In [ ]:
import os
import glob
import sys
from pathlib import Path
import psycopg2
os.environ['USE_PYGEOS'] = '0'  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import fiona
import libpysal as lps
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array
from esda import fdr
# import datashader as ds, colorcet as cc
# import holoviews as hv
# from holoviews.element.tiles import EsriImagery
# from datashader.utils import export_image
# from holoviews.operation.datashader import datashade
# hv.extension("bokeh")
import contextily as ctx
from shapely.geometry import Point, Polygon

sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import pyspace
import utils
from utils import optimize_memory_df, feature_map, show_values, sizeof_fmt, find_intersection, read_data

# Local imports
from importlib import reload  # Are you using this somewhere?


plt.rc('font', family='Helvetica')  # Try to keep configuration parameters together, maybe in a single configuration function or at the beginning of your script.
sns.set_theme(style="white")
sns.set_context("paper")

# General introduction

This notebook is dedicated to a large range of data preparation and exploratory data analyses (EDA) and including ESDA. This notebook is meant to be a building block to generate questions and problem statements that will be investigated in subsequent notebooks. 

<div class="alert alert-block alert-warning">
<b>Data</b> 

- Groupe Mutuel :
    - AOS couverture
    - LCA couverture
    - AOS prestations
    - LCA prestations
    - AOS prescriptions
    - AOS & LCA geomasked coordinates
- Socio-economic:
    - Area-based index of socio-economic position in Switzerland – Swiss-SEP (2012) https://jech.bmj.com/content/66/12/1129.long
    - Paper describing Swiss-SEP update https://smw.ch/index.php/smw/article/view/3285/5527. The new Swiss-SEP can be obtained by signing a contract with the SNC (managed by OFSP since 2022). Visit https://boris-portal.unibe.ch/handle/20.500.12422/148 for more info.
- Geographic units:
    - Lakes
    - Cantons
    - Communes
    - Populated hectares
- Accessibility
- Environmental indicators:
    - Noise pollution
    - Air pollution
    - Vegetation index
    - Land surface temperature
    
</div> 
  
<div class="alert alert-block alert-warning">
<b>TO DO LIST</b>   

- Prestations LCA:
    - One-hot encoding LCA specialties
    - Therapy-price pairs (is it solvable?)
- Prestations AOS:
    - Creation of a variable : `hospitalized` YES/NO
    - Creation of a variable : `re-hospitalized` YES/NO if hospitalized for the same MDC in a period of 6 months
    - Creation of a variable : `n_hosp` INT for the number of hospitalisations
    - One-hot encoding of DRGs
    - One-hot encoding of MDCs

- For both type of prestations and drugs `time-variables`:
    - Spike profile
    - Month with highest amount
    - Average monthly amount
    - Last 3 months amount
    - Total amount
    - Number of months above mean
    - Maximum monthly amount

- Drug AOS:
    - Create variable `polymedication` YES/NO if n_atc > 5 and age > 65 

</div> 

<div class="alert alert-block alert-success">
<b>Done</b>   

- Prepare data for PSM (yearly):
    - Preparation of prestations LCA
        - Agregation of LCA specialties
        - Total amount
        - Number of factures
    - Preparation of prestations AOS
        - One-hot encoding by sous_categorie_dispensateur
        - One-hot encoding by type_prestation
        - Total amount
        - Number of factures
        - Hospitalisation :
            - Completion of the SwissDRGs list
    - Preparation of drug AOS:
        - One-hot encoding of ATCs
        - Total number of ATCs

- Group definition (yearly) 
- Group transitions (yearly)
    
    
    
    
    

</div> 

<div class="alert alert-block alert-info">
<b>NOTES</b> 

- Basic outcomes :
    - Prestations amount
    - Prescription amount
    - Total amount
    - Amount by prestation
    - Amount by prescription
    - Polymedication (0/1) or number of different drugs
    - MIP (0/1) or number of MIP
    - Hospitalisation (0/1)
    - Number of hospitalisations
    - Total length of stay in hospital
    - Re-hospitalisation (0/1)
    - Time to re-hospitalisation
    
- Feature engineering:
    - Spike profile
    - total_monthly_drug_cost
    - total_drug_cost
    - max_monthly_drug_cost
    - mean_monthly_drug_cost
    - n_months_above_mean
    - last_3months_total_drug_cost
    - Gender
    - Age
    - Franchise
    - Canton (one-hot encoded)
    - Commune (one-hot encoded)
    - X
    - Y
    - SES (0 to 100)
    - Accessibility to healthcare services
    - Environmental indicators
    - One-hot encoded prestations (to think about...data leaks)
    - One-hot encoded prescriptions (to think about...data leaks)

- Forecasting :
    - Agregated by time -> 1 time-series
    - Agregated by time and individuals -> n individuals time-series
    - Agregated by time and groups of individuals -> n groups time-series
- Survival analyses:
    - Time to AOS usage
    - Time to LCA usage
But, we don't really have censored data! For the whole period, we could censor people who have never used any AOS or LCA. But as we see hereunder, we have a very low fraction of individuals who didn't use LCA in the period 2017-2021.
    
- Cumulated outcomes :
    - Cumulated number of bills
    - Cumulated amount of bills
    - Cumulated time hospitalized
    - Number of re-hospitalization
</div> 


# Import data

In [ ]:
# Define base data folder
data_folder  = Path('../Data/')
# Define base result folder
result_folder = Path('../Results')

## Insurance claims

In [ ]:
df_couverture_aos = read_data(data_folder/'processed'/'df_couverture_aos_preprocessed.parquet.gzip')
df_paires_lamal_lca = pd.read_csv(data_folder/'max_probs_w_zipcode_pour_david_w_uuid.csv')


In [ ]:
df_paires_lamal_lca['matching_score'].sort_values()

In [ ]:
df_paires_lamal_lca.id_lca.nunique()                                                                               

In [ ]:
# Load datasets

df_prestation_aos = read_data(data_folder/'processed'/'df_prestation_aos_preprocessed.parquet.gzip')
df_prestation_aos_cam = read_data(data_folder/'processed'/'df_prestation_aos_cam_preprocessed.parquet.gzip')
df_prestation_lca = read_data(data_folder/'processed'/'df_prestation_lca_preprocessed.parquet.gzip')
df_drug_aos = read_data(data_folder/'processed'/'df_drug_aos_preprocessed.parquet.gzip')
df_couverture_aos = read_data(data_folder/'processed'/'df_couverture_aos_preprocessed.parquet.gzip')
df_couverture_lca = read_data(data_folder/'processed'/'df_couverture_lca_preprocessed.parquet.gzip')
df_flag_aos = read_data(data_folder/'processed'/'df_flag_aos_preprocessed.parquet.gzip')

df_addresses_w_access_pollution_ndvi_lst_sep = read_data(data_folder/'processed'/'df_addresses_with_socio_env.parquet.gzip', geo=True, lon='lon_masked', lat='lat_masked')
df_full_address = read_data(data_folder/'processed'/'df_full_address.parquet.gzip', geo=True, lon='lon_masked', lat='lat_masked')

# Dictionary creation
dict_lamal_to_uuid = df_paires_lamal_lca.set_index('id_lamal')['uuid'].to_dict()
dict_lca_to_uuid = df_paires_lamal_lca.set_index('id_lca')['uuid'].to_dict()

dict_lamal_to_lca = df_paires_lamal_lca.set_index('id_lamal')['id_lca'].to_dict()
dict_lca_to_lamal = df_paires_lamal_lca.set_index('id_lca')['id_lamal'].to_dict()

In [ ]:
# df_prestation_aos_cam = read_data(data_folder/'processed'/'df_prestation_aos_cam_preprocessed.parquet.gzip')
# df_couverture_aos = read_data(data_folder/'processed'/'df_couverture_aos_preprocessed.parquet.gzip')

In [ ]:
df_prestation_aos_cam = pd.merge(df_prestation_aos_cam, df_couverture_aos[['uuid','NOANNEE','CANTON_NAME']].drop_duplicates(), left_on = ['uuid','ANNEE_TRAITEMENT'], right_on = ['uuid','NOANNEE'], how='left')

In [ ]:
dict_tarmed_cam = {0.171:'Acupuncture',
                   0.172:'Acupuncture',
                   0.173:'Acupuncture',
                   0.173:'Acupuncture',
                   0.174:'Neural therapy',
                   0.175:'Neural therapy',
                   0.176:'Neural therapy',
                   0.177:'Homeopathy',
                   0.178:'Homeopathy',
                   0.179:'Homeopathy',
                   0.180:'Homeopathy',
                   0.180:'Homeopathy',
                   0.181:'Traditional Chinese medicine',
                   0.182:'Traditional Chinese medicine',
                   0.183:'Traditional Chinese medicine',
                   0.184:'Anthroposophic medicine',
                   0.185:'Anthroposophic medicine',
                   0.186:'Anthroposophic medicine',
                   0.187:'Phytotherapy',
                   0.1871:'Phytotherapy',
                   0.1872:'Phytotherapy',
                   0.188:'Phone consultation',
                   0.189:'Phone consultation',
                   0.1895:'Phone consultation',
                   0.1896:'Phone consultation',
                   0.19:'Phone consultation',

                  }

In [ ]:
df_prestation_aos_cam['CDPOSITION_cat'] = df_prestation_aos_cam['CDPOSITION'].map(dict_tarmed_cam)

In [ ]:
df_prestation_aos_cam.CDPOSITION.unique()

In [ ]:
df_prestation_aos_cam.groupby('CANTON_NAME')['ID_DISPENSATEUR'].nunique().sort_values().plot.bar()

In [ ]:
df_prestation_aos_cam.groupby('CDPOSITION_cat')['ID_DISPENSATEUR'].nunique().sort_values().plot.bar()

## Prevalence of complementary medicine

<div class="alert alert-block alert-info">
<b>Problem Statement</b> 

What is the prevalence of integrative medicine utilization among patients in the claims data? This can help to understand the scale of the problem and the potential impact of integrative medicine interventions.

**Notes**
- What is the definition for the group that did not use LCA? Across the whole study period, within a year, a month, a treatment period?
- How do we define a treatment period?
    
    
To handle an analysis comparing two groups of individuals "Consumers of healthcare insurance" and "Non-consumers of healthcare insurance" over time, I would use a longitudinal study design. This would involve tracking the same individuals over a period of time, and recording their healthcare insurance status at multiple time points.

To handle the fact that individuals can change groups over time, I would use a combination of methods. First, I would use appropriate statistical methods, such as multi-state models, to account for the transition of individuals between the two groups over time. Second, I would ensure that the study design and data collection methods are robust enough to accurately capture changes in individuals' healthcare insurance status. Finally, I would also report any missing data or loss to follow-up in the analysis, and consider how this may have affected the results.
## Multi-state model 
A multi-state model is a type of statistical model that is used to analyze data from a longitudinal study in which individuals can transition between different states over time. In the context of healthcare insurance, a multi-state model could be used to analyze the transition of individuals between being a consumer and non-consumer of healthcare insurance over time.

A multi-state model is typically represented by a set of equations that describe the probability of transitioning between different states, and the probability of remaining in the same state. The model takes into account the time elapsed between transitions, and can include covariates, such as individual characteristics or exposures, that may influence the transition probabilities. The results of a multi-state model can provide insight into the dynamics of the healthcare insurance status over time, and inform policy decisions.
    
``` 
from lifelines import Multistate Cox Regression

# Prepare the data in the format required by the model
data = ... 

# Specify the transition matrix
transition_matrix = ...

# Fit the model
mscr = MultistateCoxRegression(data, transition_matrix)
mscr.fit()

# Print the summary
print(mscr.summary)
```
    
Or using `statsmodels` :
    
```import statsmodels.api as sm

# Prepare the data in the format required by the model
data = ... 

# Specify the transition matrix
transition_matrix = ...

# Fit the model
mscr = sm.Multinomial(data, transition_matrix)
mscr.fit()

# Print the summary
print(mscr.summary)
```

In a multi-state model, the transition matrix is a square matrix that describes the probability of transitioning between different states over time. The rows and columns of the matrix correspond to the different states, and the elements of the matrix represent the transition probabilities.

Here's an example of how you might define a transition matrix for a multi-state model with two states (consumer and non-consumer of healthcare insurance):

```
transition_matrix = [[0.9, 0.1],
                     [0.3, 0.7]]
```
This matrix represents the probability of transitioning from one state to another over time. The matrix tells us that:

The probability of staying in the consumer state is 0.9, and the probability of transitioning to the non-consumer state is 0.1
The probability of staying in the non-consumer state is 0.7, and the probability of transitioning to the consumer state is 0.3
It's important to note that the probabilities in the matrix must sum to 1 for each row, because it's representing the probability of being in a state or transitioning to another state.

You can also specify the transition matrix in a more flexible way by including covariates that may influence the transition probabilities. This can be done by defining the matrix as a function of the covariates, and estimating the parameters of the function using the data.

It's also important to note that when you use a multi-state model, you need to have the data in a format that includes the state of each individual at each time point, and the time at which the transition occurred. So, you need to have a dataset that includes the healthcare insurance status of each individual at multiple time points.

### Transition probabilities
The transition probabilities in a multi-state model are typically estimated from the data using maximum likelihood estimation (MLE) or Bayesian inference.

Maximum likelihood estimation (MLE) is a method for estimating the parameters of a statistical model that makes the observed data most probable. The MLE method estimates the transition probabilities by maximizing the likelihood of the observed data, given the model and the parameters. For example, in the lifelines package, the Multistate Cox Regression model uses MLE to estimate the transition probabilities.

Bayesian inference is a method for estimating the parameters of a statistical model based on prior information and the observed data. It uses Bayes' theorem to update the prior probability distribution of the parameters, given the data. Bayesian inference can be used to estimate the transition probabilities in a multi-state model by specifying prior distributions for the parameters and updating them using the data.

Once you have the transition probabilities calculated, you can use them to estimate the probability of an individual being in a particular state at a given time point, or the probability of transitioning from one state to another over a given time interval. These probabilities can be used to make predictions about the future healthcare insurance status of individuals and make policy decisions based on that.
    
### Example of running a multi-state model with transition probabilities estimation
```    
from lifelines import MultistateCoxRegressionFitter

# Prepare the data in the format required by the model
data = pd.DataFrame({'id': [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4],
                    'time': [1, 2, 1, 2, 3, 1, 2, 3, 1, 2, 3],
                    'state': [1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
                    'covariate': [0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]})

# Specify the transition matrix
transition_matrix = [[0, 1], [1, 0]]

# Create an instance of the Multi-State Cox Regression Fitter
mscrf = MultistateCoxRegressionFitter(transition_matrix=transition_matrix)

# Fit the model
mscrf.fit(data, 'time', 'state', id_col='id', show_progress=True)

# Print the summary
print(mscrf.summary)
```
In this example, the data is in the long format, it includes the individual ID, the time of the transition, the state of the individual (consumer or non-consumer of healthcare insurance) and a covariate that may influence the transition probability.

The transition matrix is defined as [[0, 1], [1, 0]]. The first row of the matrix represents the probability of transitioning from the consumer state to the non-consumer state, and the second row represents the probability of transitioning from the non-consumer state to the consumer state.

The MultistateCoxRegressionFitter function is used to fit the model. The function takes the data, the name of the time column, the name of the state column, and the name of the individual ID column as input. The fit() method is used to fit the model.

The summary() method prints the summary of the model, which includes the estimates of the transition probabilities and their standard errors, along with other information such as the log-likelihood, AIC, and BIC.

It's important to note that this is just an example and not a complete script, you may need to adjust it based on your data and the specific requirements of your analysis. Also, this script assumes that you have installed the lifelines package and imported the necessary library.
</div>    


### How many people having a LCA did not have any "prestations" in the period 2017-2021?

In [ ]:
# Select ID_LCA not in df_prestation_lca
insured_without_prestation = df_couverture_lca[~df_couverture_lca.ID_LCA.isin(df_prestation_lca.ID_LCA)]

# Count unique ID_LCA
unique_insured_count = insured_without_prestation.ID_LCA.nunique()

# Print the message
print(f'There are only {unique_insured_count} insured people who did not have a single LCA prestation from 2017 to 2021.')


In [ ]:
# Calculate the percentage of insured people who did not use their LCA
non_users = df_couverture_lca[~df_couverture_lca.ID_LCA.isin(df_prestation_lca.ID_LCA)].ID_LCA.nunique()
total_users = df_couverture_lca.ID_LCA.nunique()
non_user_percent = (non_users / total_users) * 100

# Print the message
print(f"That's only {non_user_percent:.3f} % of the sample. Soooo, people do like and use their LCA!")


### What about within a single year?

In [ ]:
# Filter the dataframes for 2017
df_couverture_lca_2017 = df_couverture_lca[df_couverture_lca.NOANNEE == 2017]
df_prestation_lca_2017 = df_prestation_lca[df_prestation_lca.ANNEE_TRAITEMENT == 2017]

# Find the IDs in df_couverture_lca_2017 but not in df_prestation_lca_2017
non_users_2017 = df_couverture_lca_2017[~df_couverture_lca_2017.ID_LCA.isin(df_prestation_lca_2017.ID_LCA)]

# Count the unique IDs (insured people who did not use their LCA in 2017)
non_user_count_2017 = non_users_2017.ID_LCA.nunique()

# Print the result
print(f'There are only {non_user_count_2017} insured people who did not have a single LCA prestation in 2017.')

In [ ]:
# Calculate the percentage of insured people who did not use their LCA in 2017
non_users_2017_count = non_users_2017.ID_LCA.nunique()
total_users_2017_count = df_couverture_lca_2017.ID_LCA.nunique()
non_user_percent_2017 = (non_users_2017_count / total_users_2017_count) * 100

# Print the message
print(f"That's {non_user_percent_2017:.3f} % of the sample. So people use their LCA BUT within a single year we still have more than 40% that didn't use it!")


## LCA

In [ ]:
# extract year column and ID column
df1_year = df_couverture_lca['NOANNEE']
df1_id = df_couverture_lca['uuid']
df2_year = df_prestation_lca['ANNEE_TRAITEMENT']
df2_id = df_prestation_lca['uuid']

# create a set of unique IDs in df2 for each year
df2_id_sets = df_prestation_lca.groupby('ANNEE_TRAITEMENT', observed = True)['uuid'].apply(set)

# create a new column in df1 to indicate whether the ID is present in df2 for that year
df_couverture_lca['in_df2'] = df_couverture_lca.apply(lambda x: x['uuid'] in df2_id_sets[x['NOANNEE']], axis=1)

# filter out rows where the ID is present in df2
df_no_lca_in_year = df_couverture_lca[df_couverture_lca['in_df2'] == False]
df_lca_in_year = df_couverture_lca[df_couverture_lca['in_df2'] == True]

# group by year and count the number of IDs not present in df2
count_by_year_no_lca = df_no_lca_in_year.groupby(df1_year, observed = True)['uuid'].nunique()

In [ ]:
print('Percentage of insured individuals not using their LCA within a year')
(count_by_year_no_lca/df_couverture_lca.groupby('NOANNEE', observed = True).ID_LCA.nunique()).mul(100).round(2)

Some features are directly visible:
- Almost 100% of insured individuals utilized their complementary insurance coverage during the period 2017-2021.
- Around 40% of the insured individuals do not utilize their complementary insurance coverage within a single year (for complementary medecine purposes).
- There is an increase in LCA non-utilization, most likely linked to the COVID-19 pandemic.
- The prevalence goes back to usual levels in 2021.

## AOS

In [ ]:
# extract year column and ID column
df1_year = df_couverture_aos['NOANNEE']  # all insured people
df1_id = df_couverture_aos['uuid']
df2_year = df_prestation_aos['ANNEE_TRAITEMENT']  # insured people that used aos
df2_id = df_prestation_aos['uuid']

# create a DataFrame with unique IDs in df2 for each year
df2_unique_ids = df_prestation_aos[['ANNEE_TRAITEMENT', 'uuid']].drop_duplicates()

# merge df_couverture_aos and df2_unique_ids on uuid and NOANNEE/ANNEE_TRAITEMENT columns
merged_df = pd.merge(df_couverture_aos, df2_unique_ids, left_on=['uuid', 'NOANNEE'], right_on=['uuid', 'ANNEE_TRAITEMENT'], how='left', indicator=True)

# filter out rows where the ID is present in df2
df_no_aos_in_year = optimize_memory_df(merged_df[merged_df['_merge'] == 'left_only'])
df_aos_in_year = optimize_memory_df(merged_df[merged_df['_merge'] == 'both'])

# group by year and count the number of IDs not present in df2
count_by_year_no_aos = df_no_aos_in_year.groupby(df1_year, observed=True)['uuid'].nunique()

print(count_by_year_no_aos)

### How many people having a AOS did not have any "prestations" in the period 2017-2021?

In [ ]:
# Define the set of unique ID_LAMAL values in both dataframes
unique_ids_couverture = set(df_couverture_aos['ID_LAMAL'])
unique_ids_prestation = set(df_prestation_aos['ID_LAMAL'])

# Find the difference between the sets, i.e., IDs in "df_couverture_aos" that are not in "df_prestation_aos"
ids_without_prestation = unique_ids_couverture - unique_ids_prestation

# Count the number of such IDs
num_ids_without_prestation = len(ids_without_prestation)

# Print the result
print(f'There are only {num_ids_without_prestation} insured people who did not have a single ID_LAMAL prestation from 2017 to 2021.')

In [ ]:
# Calculate the number of unique ID_LAMAL values in each dataframe
total_unique_ids = df_couverture_aos['ID_LAMAL'].nunique()
unique_ids_without_prestation = len(ids_without_prestation) # from the previous step

# Calculate the proportion of IDs that do not have a prestation
proportion_without_prestation = unique_ids_without_prestation / total_unique_ids

# Convert the proportion to a percentage and round it to three decimal places
percentage_without_prestation = round(proportion_without_prestation * 100, 3)

# Print the result
print(f"That's only {percentage_without_prestation} % of the sample. So, there are more people using their LCA than using their LAMAL!")

### What about within a single year?

In [ ]:
df_unique_uuid_by_year = df_prestation_aos_cam.groupby(['ANNEE_TRAITEMENT']).uuid.nunique().reset_index()
print("Number of distinct individuals receiving CAM within AOS by year",df_unique_uuid_by_year)

In [ ]:
print("Total des prestations CAM within the AOS for the period 2017-2021: ",df_prestation_aos_cam.PRESTATIONS_BRUTES.sum().round(0))

In [ ]:
%%time
# extract year column and ID column
df1_year = df_couverture_aos['NOANNEE']  # all insured people
df1_id = df_couverture_aos['uuid']
df2_year = df_prestation_aos['ANNEE_TRAITEMENT']  # insured people that used aos
df2_id = df_prestation_aos['uuid']

# create a DataFrame with unique IDs in df2 for each year
df2_unique_ids = df_prestation_aos[['ANNEE_TRAITEMENT', 'uuid']].drop_duplicates()

# merge df_couverture_aos and df2_unique_ids on uuid and NOANNEE/ANNEE_TRAITEMENT columns
merged_df = pd.merge(df_couverture_aos, df2_unique_ids, left_on=['uuid', 'NOANNEE'], right_on=['uuid', 'ANNEE_TRAITEMENT'], how='left', indicator=True)

# filter out rows where the ID is present in df2
df_no_aos_in_year = merged_df[merged_df['_merge'] == 'left_only']
df_aos_in_year = merged_df[merged_df['_merge'] == 'both']
df_aos_in_year = optimize_memory_df(df_aos_in_year)

# group by year and count the number of IDs not present in df2
count_by_year_no_aos = df_no_aos_in_year.groupby(df1_year, observed=True)['uuid'].nunique()
print('Number of people without AOS claims within a year')
print(count_by_year_no_aos)

In [ ]:
print('Percentage of insured individuals not using their LCA within a year')

count_by_year_no_aos/df_couverture_aos.groupby('NOANNEE', observed = True).uuid.nunique()

Some features are directly visible:
- 98% of insured individuals utilized their mandatory insurance coverage (AOS) during the period 2017-2021.
- Around 5-10% of the insured individuals do not utilize their mandatory insurance coverage (AOS) within a year (for complementary medecine purposes).
- There seems to be a trend of increasing prevalence of AOS usage over time.
- The prevalence of AOS utilization is the highest in 2021, with only around 2% not utilizing it.

### Verification of total amounts and number of users

In [ ]:
print('CAM - AOS', df_prestation_aos_cam['PRESTATIONS_BRUTES'].sum()/1000000, df_prestation_aos_cam.ID_LAMAL.nunique())
print('AOS', df_prestation_aos['PRESTATIONS_BRUTES'].sum()/1000000, df_prestation_lca.ID_LAMAL.nunique())
print('LCA', df_prestation_lca['PRESTATIONS_BRUTES'].sum()/1000000, df_prestation_lca.ID_LCA.nunique())

Everything ok at this stage !

### People not insured for both insurance each year of the 2017-2021 period
After some digging, we found that we have a good number of people who actually don't have both insurance for the whole period ! 


In [ ]:
df_not_insured_aos_in_year = pd.merge(df_couverture_lca[["uuid",'NOANNEE']].drop_duplicates(), df_couverture_aos[["uuid",'NOANNEE','Annee_naiss']].drop_duplicates(), on = ['uuid','NOANNEE'], how = 'left')
df_not_insured_lca_in_year = pd.merge(df_couverture_aos[["uuid",'NOANNEE']].drop_duplicates(), df_couverture_lca[["uuid",'NOANNEE','Annee_naiss']].drop_duplicates(), on = ['uuid','NOANNEE'], how = 'left')

In [ ]:
df_not_insured_aos_in_year = df_not_insured_aos_in_year[df_not_insured_aos_in_year.Annee_naiss.isnull()]
df_not_insured_lca_in_year = df_not_insured_lca_in_year[df_not_insured_lca_in_year.Annee_naiss.isnull()]

This technique is great to identify people that present in one type of insurance but not the other (not insured that year). But can't identify people that are in neither...not insured at all. Forming a gp9 based on not_insured_aos and not_insured_lca won't work bc of that (I commented out this code below). What we can easily do is by elimination define that these are the ones not insured at all.

## Define groups

<div class="alert alert-block alert-info">
<b>Problem Statement</b> 
    
We need to create groups of insured individuals based on their AOS and LCA utilization. This step is rather straightforward. The only challenge is to decide how to consider how individuals may change groups overtime depending on the time period selected. Are we ok with the fact that individuals might be part of group 1 in year 1, then 3 in year 2, then 4 in year 3, then 1 again in year 4, etc. 
    
 If we consider that insurance utilization is independent between years, I would say yes. But that is a simplification, as we know that it is not actually independent as there is autocorrelation in an individual's healthcare consumption.
    
We can :
1) Calculate the prevalence of individuals switching groups over the whole period and over any two years. This will help better understand how people change behaviors over time and indicate how much back and forth there might be between AOS and LCA. 
2) We can use models for longitudinal data (General Estimating Equation GEE, Multilevel modeling) using the UUID as a random variable.
3) See if we can predict usage of LCA. Are there any condition, period of the year, or sociodemographic determinants of utilization?
4) See if we can predict when someone is going to use LCA ! That would be a great forecasting task
    
- Define groups : 
    1) Group 1 : Used both LCA & LAMAL
    2) Group 2 : Used only LCA
    3) Group 3 : Used only LAMAL
    4) Group 4 : Used neither
- Time period : whole period, year, quarter, month, treatment
</div>    

In [ ]:
# df_couverture_aos['uuid'] = df_couverture_aos['ID_LAMAL'].map(dict_lamal_to_uuid)
# df_couverture_lca['uuid'] = df_couverture_lca['ID_LCA'].map(dict_lca_to_uuid)

In [ ]:
# df_prestation_aos['uuid'] = df_prestation_aos['ID_LAMAL'].map(dict_lamal_to_uuid)
# df_prestation_lca['uuid'] = df_prestation_lca['ID_LCA'].map(dict_lca_to_uuid)

### Simplest version - Time period = year

In [ ]:
s1 = df_no_aos_in_year.groupby('NOANNEE', observed = True).uuid.unique()
s2 = df_aos_in_year.groupby('NOANNEE', observed = True).uuid.unique()
s3 = df_lca_in_year.groupby('NOANNEE', observed = True).uuid.unique()
s4 = df_no_lca_in_year.groupby('NOANNEE', observed = True).uuid.unique()
s5 = df_not_insured_aos_in_year.groupby('NOANNEE',observed = True).uuid.unique()
s6 = df_not_insured_lca_in_year.groupby('NOANNEE',observed = True).uuid.unique()
s7 = df_prestation_aos_cam.groupby('ANNEE_TRAITEMENT', observed = True).uuid.unique()

In [ ]:
df_gps = pd.concat([s1, s2, s3, s4, s5, s6, s7], axis = 1)
df_gps.columns =  ['no_aos', 'aos', 'lca', 'no_lca', 'not_insured_aos', 'not_insured_lca', 'cam_in_aos']

In [ ]:
def _find_intersection(row, col1, col2):
    return list(set(row[col1]).intersection(row[col2]))

In [ ]:
# Apply the function to each row of the DataFrame
df_gps['gp1'] = df_gps.apply(lambda x: find_intersection(x, ['aos', 'lca']), axis=1)
df_gps['gp2'] = df_gps.apply(lambda x: find_intersection(x, ['no_aos', 'lca']), axis=1)
df_gps['gp3'] = df_gps.apply(lambda x: find_intersection(x, ['aos', 'no_lca']), axis=1)
df_gps['gp4'] = df_gps.apply(lambda x: find_intersection(x, ['no_aos', 'no_lca']), axis=1)
df_gps['gp5'] = df_gps.apply(lambda x: find_intersection(x, ['not_insured_aos', 'lca']), axis=1)
df_gps['gp6'] = df_gps.apply(lambda x: find_intersection(x, ['aos', 'not_insured_lca']), axis=1)
df_gps['gp7'] = df_gps.apply(lambda x: find_intersection(x, ['no_aos', 'not_insured_lca']), axis=1)
df_gps['gp8'] = df_gps.apply(lambda x: find_intersection(x, ['not_insured_aos', 'no_lca']), axis=1)
df_gps['gp9'] = df_gps.apply(lambda x: find_intersection(x, ['aos', 'cam_in_aos']), axis=1)
df_gps['gp10'] = df_gps.apply(lambda x: find_intersection(x, ['no_aos', 'cam_in_aos']), axis=1)
df_gps['gp11'] = df_gps.apply(lambda x: find_intersection(x, ['lca', 'cam_in_aos']), axis=1)
df_gps['gp12'] = df_gps.apply(lambda x: find_intersection(x, ['no_lca', 'cam_in_aos']), axis=1)
df_gps['gp13'] = df_gps.apply(lambda x: find_intersection(x, ['lca', 'aos', 'cam_in_aos']), axis=1)

In [ ]:
dict_labels_gps =  {1:'LCA & AOS',
                    2:'LCA only',
                    3:'AOS only',
                    4:'No usage',
                    5:'Not insured AOS, LCA',
                    6:'Not insured LCA, AOS',
                    7:'No usage - No AOS & not insured LCA',
                    8:'No usage - No LCA & not insured AOS',
                    9:'AOS & CAM in AOS',
                    10:'No AOS & CAM in AOS',
                    11:'LCA & CAM in AOS',
                    12:'No LCA & CAM in AOS',
                    13:'LCA & AOS & CAM in AOS'}

In [ ]:
for i in range(1, 14):
    df_gps[f'len_gp{i}'] = df_gps[f'gp{i}'].apply(len)

In [ ]:
df_gps = df_gps.reset_index()

In [ ]:
df_gps_exploded = pd.concat([pd.DataFrame({'NOANNEE':df_gps['index'],'uuid': df_gps[col].explode(), 'gp': int(col.replace('gp',''))}) for col in df_gps.columns if col.startswith('gp')]).reset_index(drop=True)

In [ ]:
# We retain only the gps that are non-overlapping. 
# Groups 9, 10, 11, 12, 13 are a bit problematic but may be of interest for other analyses
df_gps_exploded_excl_gp = df_gps_exploded[df_gps_exploded.gp < 9]

In [ ]:
# gp_mapping_dict = df_gps_exploded.set_index(['NOANNEE','uuid'])['gp'].to_dict()

In [ ]:
# def assign_value(row, col1 , col2):
#     col1_val = row[col1]
#     col2_val = row[col2]
#     return gp_mapping_dict.get((col1_val, col2_val))

In [ ]:
df_couverture_aos = df_couverture_aos.merge(df_gps_exploded_excl_gp, on=["NOANNEE", "uuid"], how="left")
df_couverture_lca = df_couverture_lca.merge(df_gps_exploded_excl_gp, on=["NOANNEE", "uuid"], how="left")
df_prestation_lca = df_prestation_lca.merge(df_gps_exploded_excl_gp, left_on=['ANNEE_TRAITEMENT','uuid'], right_on=["NOANNEE", "uuid"], how="left")
df_prestation_aos = df_prestation_aos.merge(df_gps_exploded_excl_gp, left_on=['ANNEE_TRAITEMENT','uuid'], right_on=["NOANNEE", "uuid"], how="left")

In [ ]:
df_prestation_aos_cam = df_prestation_aos_cam.merge(df_gps_exploded, left_on=['ANNEE_TRAITEMENT','uuid'], right_on=["NOANNEE", "uuid"], how="left")

In [ ]:
df_prestation_aos_cam.gp.value_counts()

## Feature engineering

### Prestations AOS

Create a dataset containing the outcomes relevant to the study by month and by year

- Nombre de prestations par an et par patient
- Montant total (BRUT) de prestations par patient et par an
- Montant total (NET) de prestations par patient et par an
- Nombre de factures par an et par patient

In [ ]:
df_prestation_aos[['ANNEE_TRAITEMENT','MOIS_TRAITEMENT']] = df_prestation_aos[['ANNEE_TRAITEMENT','MOIS_TRAITEMENT']].astype('category')

In [ ]:
# For time series analyses, or month based analyses in the future

if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'n_prestation_aos_by_patient_by_month.parquet.gzip'):
    n_prestation_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).size()
else:
    n_prestation_aos_by_patient_by_month = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'n_prestation_aos_by_patient_by_month.parquet.gzip')

if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'sum_prestation_brutes_aos_by_patient_by_month.parquet.gzip'):
    sum_prestation_brutes_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).PRESTATIONS_BRUTES.sum()
else:
    sum_prestation_brutes_aos_by_patient_by_month = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'sum_prestation_brutes_aos_by_patient_by_month.parquet.gzip')

if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'sum_prestation_nettes_aos_by_patient_by_month.parquet.gzip'):
    sum_prestation_nettes_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).PRESTATIONS_NETTES.sum()
else:
    sum_prestation_nettes_aos_by_patient_by_month = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'sum_prestation_nettes_aos_by_patient_by_month.parquet.gzip') 

if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'n_bill_prestation_aos_by_patient_by_month.parquet.gzip'):
    n_bill_prestation_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).NBRE_FACTURES.sum()
else:
    n_bill_prestation_aos_by_patient_by_month = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'n_bill_prestation_aos_by_patient_by_month.parquet.gzip')    

In [ ]:
# # For time series analyses, or month based analyses in the future
# n_prestation_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).size()
# sum_prestation_brutes_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).PRESTATIONS_BRUTES.sum()
# sum_prestation_nettes_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).PRESTATIONS_NETTES.sum()
# n_bill_prestation_aos_by_patient_by_month = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).NBRE_FACTURES.sum()

In [ ]:
# Compute key indicators from AOS spending
n_prestation_aos_by_patient_by_year = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT']).size()
sum_prestation_brutes_aos_by_patient_by_year = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT']).PRESTATIONS_BRUTES.sum()
sum_prestation_brutes_aos_ambulatoire_by_patient_by_year = df_prestation_aos[df_prestation_aos.TYPE_PRESTATION == 'AMBULATOIRE'].groupby(['uuid','ANNEE_TRAITEMENT']).PRESTATIONS_BRUTES.sum()
sum_prestation_brutes_aos_stationnaire_by_patient_by_year = df_prestation_aos[df_prestation_aos.TYPE_PRESTATION == 'STATIONNAIRE'].groupby(['uuid','ANNEE_TRAITEMENT']).PRESTATIONS_BRUTES.sum()
sum_prestation_nettes_aos_by_patient_by_year = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT']).PRESTATIONS_NETTES.sum()
n_ss_categorie_disp_aos_by_patient_by_year = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT']).SOUS_CATEGORIE_DISPENSATEUR.nunique()
n_bill_prestation_aos_by_patient_by_year = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT']).NBRE_FACTURES.sum()
sum_prestation_brutes_cam_aos_by_patient_by_year = df_prestation_aos_cam.groupby(['uuid','ANNEE_TRAITEMENT']).PRESTATIONS_BRUTES.sum()

In [ ]:
sum_prestation_brutes_aos_by_patient_by_year.sum()/1000000

In [ ]:
sum_prestation_brutes_aos_by_patient_by_month = pd.DataFrame(sum_prestation_brutes_aos_by_patient_by_month).reset_index()
sum_prestation_brutes_aos_by_patient_by_month['MOIS_TRAITEMENT'] = sum_prestation_brutes_aos_by_patient_by_month['MOIS_TRAITEMENT'].astype(float)
sum_prestation_brutes_aos_by_patient_by_year = pd.DataFrame(sum_prestation_brutes_aos_by_patient_by_year)

In [ ]:
# Step 1: Calculate the cumulative sum within each year
sum_prestation_brutes_aos_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] = sum_prestation_brutes_aos_by_patient_by_month.sort_values(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].cumsum()

month_reached_1000_aos = sum_prestation_brutes_aos_by_patient_by_month[sum_prestation_brutes_aos_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 1000].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()
month_reached_5000_aos = sum_prestation_brutes_aos_by_patient_by_month[sum_prestation_brutes_aos_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 5000].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()
month_reached_10000_aos = sum_prestation_brutes_aos_by_patient_by_month[sum_prestation_brutes_aos_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 10000].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()

In [ ]:
# Highest month
max_monthly_prestation_cost = sum_prestation_brutes_aos_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].max()

In [ ]:
# Monthly average
mean_monthly_prestation_cost = sum_prestation_brutes_aos_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].mean()

In [ ]:
# Mean three last months
last_3months_total_prestation_cost = sum_prestation_brutes_aos_by_patient_by_month[sum_prestation_brutes_aos_by_patient_by_month.MOIS_TRAITEMENT.isin([10,11,12])].groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].sum()

In [ ]:
# N months above mean - LONG
# n_months_above_mean_prestation = sum_prestation_brutes_aos_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT'], group_keys=True)['PRESTATIONS_BRUTES'].apply(lambda x: x> x.mean())
# n_months_above_mean_prestation = n_months_above_mean_prestation[n_months_above_mean_prestation == True].groupby(['uuid','ANNEE_TRAITEMENT']).size()

In [ ]:
# Spike profile - LONG
# spike_profile_prestation = sum_prestation_brutes_aos_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT'], group_keys=True)['PRESTATIONS_BRUTES'].apply(lambda x: x> 0.50*x.sum())
# spike_profile_prestation = spike_profile_prestation[spike_profile_prestation == True].groupby(['uuid','ANNEE_TRAITEMENT']).size()

#### Nombre de categorie dispensateur

In [ ]:
n_dispensateur_aos = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT']).CATEGORIE_DISPENSATEUR.apply(set).apply(lambda x: len(x) if isinstance(x, set) else 0)

### Concat all AOS features

In [ ]:
df_prestation_aos_outcomes_by_year = pd.concat([n_prestation_aos_by_patient_by_year, sum_prestation_brutes_aos_by_patient_by_year, sum_prestation_brutes_cam_aos_by_patient_by_year, sum_prestation_brutes_aos_ambulatoire_by_patient_by_year, sum_prestation_brutes_aos_stationnaire_by_patient_by_year, sum_prestation_nettes_aos_by_patient_by_year, n_bill_prestation_aos_by_patient_by_year,month_reached_1000_aos, month_reached_5000_aos, month_reached_10000_aos, max_monthly_prestation_cost,mean_monthly_prestation_cost, last_3months_total_prestation_cost, n_dispensateur_aos, n_ss_categorie_disp_aos_by_patient_by_year], axis = 1).reset_index().rename(columns = {0:'NBROWS'})
# df_prestation_aos_outcomes_by_year = df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT'], observed = True)[['NBROWS','PRESTATIONS_BRUTES','PRESTATIONS_NETTES','NBRE_FACTURES']].sum().reset_index()

In [ ]:
df_prestation_aos_outcomes_by_year.columns = ['uuid', 'ANNEE_TRAITEMENT', 'NBROWS', 'PRESTATIONS_BRUTES','PRESTATIONS_BRUTES_CAM', 'PRESTATIONS_BRUTES_AMBULATOIRE', "PRESTATIONS_BRUTES_STATIONNAIRE",
       'PRESTATIONS_NETTES', 'NBRE_FACTURES','month_reached_1000_aos','month_reached_5000_aos','month_reached_10000_aos','max_monthly_aos','mean_monthly_aos','last3month_aos','n_cat_dispensateur_aos','n_ss_cat_dispensateur_aos']

In [ ]:
# df_prestation_aos_outcomes['treatmentdate'] = pd.to_datetime(df_prestation_aos_outcomes['ANNEE_TRAITEMENT'].astype('string') + '-' + df_prestation_lca_outcomes['MOIS_TRAITEMENT'].astype('string'))
# df_prestation_aos_outcomes['treatmentmonth'] = df_prestation_aos_outcomes['treatmentdate'].dt.strftime('%Y-%m')

#### Amount CAM by CDPOSITION

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_CDPOSITION.parquet.gzip'):

    df_amount_by_cdposition = pd.DataFrame(df_prestation_aos_cam.groupby(['uuid','ANNEE_TRAITEMENT','CDPOSITION_cat']).PRESTATIONS_BRUTES.sum()).reset_index()
    df_amount_by_cdposition = df_amount_by_cdposition.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'CDPOSITION_cat', values = 'PRESTATIONS_BRUTES').reset_index().fillna(0)
    df_amount_by_cdposition.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_cdposition.parquet.gzip', compression = 'gzip')
else:
    df_amount_by_cdposition = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_cdposition.parquet.gzip')

#### Amount by type sinistre

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_sinistre.parquet.gzip'):

    df_amount_by_sinistre = pd.DataFrame(df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','CDTYPESINISTRE']).PRESTATIONS_BRUTES.sum()).reset_index()
    df_amount_by_sinistre = df_amount_by_sinistre.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'CDTYPESINISTRE', values = 'PRESTATIONS_BRUTES').reset_index()
    df_amount_by_sinistre.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_sinistre.parquet.gzip', compression = 'gzip')
else:
    df_amount_by_sinistre = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_sinistre.parquet.gzip')

#### Amount by sous-catégorie dispensateur

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_souscat_disp.parquet.gzip'):

    df_amount_by_souscat_disp = pd.DataFrame(df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','SOUS_CATEGORIE_DISPENSATEUR']).PRESTATIONS_BRUTES.sum()).reset_index()
    df_amount_by_souscat_disp = df_amount_by_souscat_disp.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'SOUS_CATEGORIE_DISPENSATEUR', values = 'PRESTATIONS_BRUTES').reset_index()
    df_amount_by_souscat_disp.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_souscat_disp.parquet.gzip', compression = 'gzip')
else:
    df_amount_by_souscat_disp = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_souscat_disp.parquet.gzip')

#### Proportion by type of prestation

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_type_prestation.parquet.gzip'):
    df_amount_by_type_prestation = pd.DataFrame(df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','TYPE_PRESTATION'])['PRESTATIONS_BRUTES'].sum()).reset_index()
    df_amount_by_type_prestation = df_amount_by_type_prestation.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'TYPE_PRESTATION', values = 'PRESTATIONS_BRUTES').reset_index()
    df_amount_by_type_prestation['AMBULATOIRE'] = df_amount_by_type_prestation['AMBULATOIRE']
    df_amount_by_type_prestation['STATIONNAIRE'] = df_amount_by_type_prestation['STATIONNAIRE']
    df_amount_by_type_prestation.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_type_prestation.parquet.gzip', compression = 'gzip')
else:
    df_amount_by_type_prestation = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_type_prestation.parquet.gzip')

In [ ]:
# df_prestation_aos_outcomes_by_year = pd.concat([n_prestation_aos_by_patient_by_year, sum_prestation_brutes_aos_by_patient_by_year, sum_prestation_nettes_aos_by_patient_by_year, n_bill_prestation_aos_by_patient_by_year], axis = 1).reset_index().rename(columns = {0:'NBROWS'})

### SwissDRGs

1. Manually copy pasted list from [SwissDRGs](https://datenspiegel120.swissdrg.org/drgs?locale=fr) into an excel file.

2. Completed the file because there were missing DRGs, we used [SwissDRG v.6.0](https://www.swissdrg.org/application/files/7114/8111/2947/Swiss-DRG_Version_6.0_Fallpauschalenkatalog_franz_KV_genehmigt.pdf), [SwissDRG v.7.0](https://www.swissdrg.org/application/files/9514/9907/7413/Swiss-DRG_Version_7.0_Fallpauschalenkatalog_PV_2016_2018_genehmigt_f.pdf) and [SwissDRG v.12.0](https://www.swissdrg.org/application/files/5016/7152/9703/SwissDRG-Version_12.0_Fallpauschalenkatalog_AV_2023_2023_f.pdf) 


We can also download excel version of each SwissDRG version [Here](https://www.swissdrg.org/fr/somatique-aigue/archiv-swissdrg-system/systeme-swissdrg-1002021)

In [ ]:
df_swiss_drg = pd.read_excel(data_folder/'raw'/'SwissDRGs.xlsx')
df_swiss_drg_v10 = pd.read_excel(data_folder/'raw'/'SwissDRG-Version_10.0.xlsx', sheet_name = 'Hôpitaux de soins aigus')

In [ ]:
df_swiss_drg_v10_supplemented = pd.concat([df_swiss_drg_v10, df_swiss_drg[df_swiss_drg.DRG.isin(df_swiss_drg_v10.DRG) == False]])

In [ ]:
df_swiss_drg_v10_supplemented = df_swiss_drg_v10_supplemented.rename(columns = {'Désignation':'DRG_desc'})

In [ ]:
df_swiss_drg_v10_supplemented[df_swiss_drg_v10_supplemented['DRG_desc'].str.contains('\?', na = False)]

In [ ]:
df_prestation_aos_drg = pd.merge(df_prestation_aos, df_swiss_drg_v10_supplemented[['DRG','DRG_desc','Cost-weight','Durée moyenne de séjour']], left_on = 'CODE_DRG', right_on = 'DRG', how = 'left')

In [ ]:
df_prestation_aos_drg.to_parquet(data_folder/'processed'/'df_aos_drg.parquet.gzip', compression='gzip')

In [ ]:
del df_prestation_aos_drg

In [ ]:
# df_prestation_aos = df_prestation_aos.drop(['DRG_desc','Cost-weight','Durée moyenne de séjour'],axis = 1)

#### TO DO : There seems to be something missing here...We are not done with DRGs 

#### Hospitalized

Flags stationnaires : 'LOCDRHOSP' : flag séjour officiel de la compensation des risques : si 1, l’assuré a passé au moins 3 nuits consécutives dans 1 hopital ou 1 EMS.
'LOPCG' : PCG oui/non, est à 1 si l’assuré à au moins 1 PCG. Ci-dessous aussi est à 1 si l’assuré à le PCG.

In [ ]:
df_nomenclature_flags = pd.read_csv(data_folder/'raw'/'GM'/'Full'/'Nomenclature_flags.csv')

In [ ]:
df_hospital = df_prestation_aos[df_prestation_aos.CODE_DRG.isnull()==False].groupby(['uuid','ANNEE_TRAITEMENT'])['CODE_DRG'].transform(lambda x: x.notnull().any())

In [ ]:
df_prestation_aos[~df_prestation_aos.CODE_DRG.isnull()]['CODE_DRG']

In [ ]:
df_inpatient_hosp = df_prestation_aos[(df_prestation_aos.CODE_DRG.isnull()==False)&(df_prestation_aos.TYPE_PRESTATION == 'STATIONNAIRE')]
df_outpatient_hosp = df_prestation_aos[(df_prestation_aos.CODE_DRG.isnull()==False)&(df_prestation_aos.TYPE_PRESTATION == 'AMBULATOIRE')]

df_outpatient_hosp['treatmentmonth'] = pd.to_datetime(df_outpatient_hosp['treatmentmonth'].astype('string'))
df_inpatient_hosp['treatmentmonth'] = pd.to_datetime(df_inpatient_hosp['treatmentmonth'].astype('string'))

#### Number of hospitalisations

- We differentiate in and outpatient hosp.
- We consider that if there is an hosp. for the same DRGs for two consecutive months, it counts for 1 hosp.

In [ ]:
df_grouped_month_inpatient = pd.DataFrame(df_prestation_aos[(df_prestation_aos.CODE_DRG.isnull()==False)&(df_prestation_aos.TYPE_PRESTATION == 'STATIONNAIRE')].groupby(['uuid', 'treatmentmonth']).CODE_DRG.apply(set)).reset_index()
df_grouped_month_outpatient = pd.DataFrame(df_prestation_aos[(df_prestation_aos.CODE_DRG.isnull()==False)&(df_prestation_aos.TYPE_PRESTATION == 'AMBULATOIRE')].groupby(['uuid', 'treatmentmonth']).CODE_DRG.apply(set)).reset_index()

In [ ]:
df_grouped_month_inpatient['treatmentmonth'] = pd.to_datetime(df_grouped_month_inpatient['treatmentmonth'].astype('string'))
df_grouped_month_outpatient['treatmentmonth'] = pd.to_datetime(df_grouped_month_outpatient['treatmentmonth'].astype('string'))

In [ ]:
# DOES NOT WORK AS INTENDED BUT I KEEP IT BC IT MAY BE USEFUL FOR TIME BETWEEN HOSP

# # Group the data by individual ID and year, and count the number of events within a year having distinct DRGs
# df_grouped_year_inpatient = df_grouped_month_inpatient.groupby(['uuid', pd.Grouper(key='treatmentmonth', freq='1Y')])['CODE_DRG'].apply(lambda x: x.ne(x.shift()).cumsum().nunique() if not x.dropna().empty else 0)
# df_grouped_year_outpatient = df_grouped_month_outpatient.groupby(['uuid', pd.Grouper(key='treatmentmonth', freq='1Y')])['CODE_DRG'].apply(lambda x: x.ne(x.shift()).cumsum().nunique() if not x.dropna().empty else 0)

# # Reset the index to get a new dataframe with the 'uuid', 'treatmentdate', and 'num_hospitalisations' columns
# df_result_inpatient = df_grouped_year_inpatient.reset_index(name='num_hospitalisations')
# df_result_outpatient = df_grouped_year_outpatient.reset_index(name='num_hospitalisations')

# # Merge the 'num_hospitalisations' column back to the original dataframe
# # df = pd.merge(df, df_result, on=['uuid', 'treatmentdate'], how='left')

# # Replace NaN values with 0
# # df['num_hospitalisations'] = df['num_hospitalisations'].fillna(0)

# # Convert the 'num_hospitalisations' column to integer type
# # df['num_hospitalisations'] = df['num_hospitalisations'].astype(int)

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_grouped_year_inpatient.parquet.gzip'):
    
    df_grouped_month_inpatient['year'] = df_grouped_month_inpatient['treatmentmonth'].dt.year

    groups = df_grouped_month_inpatient.groupby(['uuid', pd.Grouper(key='treatmentmonth', freq='1Y')])

    for (uuid, month), group in groups:
        # initialize the count of hospitalisations
        count = 0
        # initialize the previous month
        prev_month = ''
        prev_code = ''
        # loop over the rows in the group
        for i, row in group.iterrows():
            # check if there is a hospitalisation code
            if not pd.isna(row['CODE_DRG']):
                # check if it's a new hospitalisation event
                if row['treatmentmonth'] != prev_month:
                    if row['CODE_DRG'] != prev_code:
                        count += 1
                        prev_month = row['treatmentmonth']
                        prev_code = row['CODE_DRG']
            # update the count of hospitalisations for this row
            df_grouped_month_inpatient.loc[i, 'n_inpatient_hosp'] = count

    df_grouped_year_inpatient = pd.DataFrame(df_grouped_month_inpatient.groupby(['uuid','year'])['n_inpatient_hosp'].max()).reset_index()
    df_grouped_year_inpatient.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_grouped_year_inpatient.parquet.gzip', compression = 'gzip')
else:
    df_grouped_year_inpatient = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_grouped_year_inpatient.parquet.gzip')

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_grouped_year_outpatient.parquet.gzip'):

    df_grouped_month_outpatient['year'] = df_grouped_month_outpatient['treatmentmonth'].dt.year

    groups = df_grouped_month_outpatient.groupby(['uuid', pd.Grouper(key='treatmentmonth', freq='1Y')])

    for (uuid, month), group in groups:
        # initialize the count of hospitalisations
        count = 0
        # initialize the previous month
        prev_month = ''
        prev_code = ''
        # loop over the rows in the group
        for i, row in group.iterrows():
            # check if there is a hospitalisation code
            if not pd.isna(row['CODE_DRG']):
                # check if it's a new hospitalisation event
                if row['treatmentmonth'] != prev_month:
                    if row['CODE_DRG'] != prev_code:
                        count += 1
                        prev_month = row['treatmentmonth']
                        prev_code = row['CODE_DRG']
            # update the count of hospitalisations for this row
            df_grouped_month_outpatient.loc[i, 'n_outpatient_hosp'] = count

    df_grouped_year_outpatient = pd.DataFrame(df_grouped_month_outpatient.groupby(['uuid','year'])['n_outpatient_hosp'].max()).reset_index()
    df_grouped_year_outpatient.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_grouped_year_outpatient.parquet.gzip', compression = 'gzip')
else:
    df_grouped_year_outpatient = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_grouped_year_outpatient.parquet.gzip')

#### Number of hospitalized months

In [ ]:
df_n_month_outpatient = pd.DataFrame(df_grouped_month_outpatient[df_grouped_month_outpatient.CODE_DRG.isnull()==False].groupby(['uuid', pd.Grouper(key='treatmentmonth', freq='1Y')]).size(), columns = ['n_month_outpatienthosp']).reset_index()
df_n_month_inpatient = pd.DataFrame(df_grouped_month_inpatient[df_grouped_month_inpatient.CODE_DRG.isnull()==False].groupby(['uuid', pd.Grouper(key='treatmentmonth', freq='1Y')]).size(), columns = ['n_month_inpatienthosp']).reset_index()

In [ ]:
df_n_month_outpatient['year'] = df_n_month_outpatient['treatmentmonth'].dt.year
df_n_month_inpatient['year'] = df_n_month_inpatient['treatmentmonth'].dt.year

In [ ]:
df_grouped_month_inpatient['year'] = df_grouped_month_inpatient['treatmentmonth'].dt.year
df_grouped_month_outpatient['year'] = df_grouped_month_outpatient['treatmentmonth'].dt.year

Upon evaluating certain aspects, here are some conclusions:

- The procedure to compute the N months hosp is correct
- Most individuals have an equal annual number of months for outpatient and inpatient (risk of multicollinearity)
- 

#### Time to rehospitalisation

In [ ]:
df_time_to_rehosp_outpatient = (df_grouped_month_outpatient[df_grouped_month_outpatient.CODE_DRG.isnull()==False].groupby('uuid')['treatmentmonth'].diff().fillna(pd.Timedelta(seconds=0))
                                   / pd.Timedelta(days=30)).astype(int)
df_time_to_rehosp_inpatient = (df_grouped_month_inpatient[df_grouped_month_inpatient.CODE_DRG.isnull()==False].groupby('uuid')['treatmentmonth'].diff().fillna(pd.Timedelta(seconds=0))
                                   / pd.Timedelta(days=30)).astype(int)

In [ ]:
df_grouped_month_outpatient['time_to_rehosp_out'] = df_time_to_rehosp_outpatient
df_grouped_month_inpatient['time_to_rehosp_in'] = df_time_to_rehosp_inpatient

In [ ]:
df_grouped_month_inpatient['year'] = df_grouped_month_inpatient['treatmentmonth'].dt.year
df_grouped_month_outpatient['year'] = df_grouped_month_outpatient['treatmentmonth'].dt.year

In [ ]:
df_time_to_rehosp_inpatient_final = df_grouped_month_inpatient[df_grouped_month_inpatient.time_to_rehosp_in > 0].dropna(subset = 'time_to_rehosp_in').drop(['treatmentmonth','CODE_DRG'], axis = 1).groupby(['uuid','year']).mean().reset_index()
df_time_to_rehosp_outpatient_final = df_grouped_month_outpatient[df_grouped_month_outpatient.time_to_rehosp_out > 0].dropna(subset = 'time_to_rehosp_out').drop(['treatmentmonth','CODE_DRG'], axis = 1).groupby(['uuid','year']).mean().reset_index()

In [ ]:
df_time_to_rehosp_inpatient_final[df_time_to_rehosp_inpatient_final.time_to_rehosp_in == 0]

## Concat hospilisations features

In [ ]:
df_outcomes_hosp = pd.merge(df_grouped_year_inpatient, df_grouped_year_outpatient, on = ['uuid','year'], how = 'outer').fillna(0)
df_outcomes_hosp = pd.merge(df_outcomes_hosp, df_n_month_outpatient[['uuid','year','n_month_outpatienthosp']], on = ['uuid','year'], how = 'outer').fillna(0)
df_outcomes_hosp = pd.merge(df_outcomes_hosp, df_n_month_inpatient[['uuid','year','n_month_inpatienthosp']], on = ['uuid','year'], how = 'outer').fillna(0)
df_outcomes_hosp = pd.merge(df_outcomes_hosp, df_time_to_rehosp_inpatient_final[['uuid','year','time_to_rehosp_in']], on = ['uuid','year'], how = 'outer')
df_outcomes_hosp = pd.merge(df_outcomes_hosp, df_time_to_rehosp_outpatient_final[['uuid','year','time_to_rehosp_out']], on = ['uuid','year'], how = 'outer')

## (FIXED) Identify the CAM in the AOS

**Strategy** : So...there are four types of CAM in the LAMal, homeopathy, MTC, phytotherapy and anthroposophic medicine. These have to be performed by certified doctors to be reimbursed. So, how do we go about this?

1. Most of this must be in the from of prescriptions? -> Find prescriptions in these 4 but...we can't retrieve the doctor that prescribed since it agregated by sous_categorie dispensateur for the prestations.
2. ATC codes related to CAM?
3. 

### UPDATE : Data extracted based on DRGs corresponding to CAM in LAMAL sent by C. Bagnoud

### LCA

Create a dataset containing the outcomes relevant to the study by month and by year

- Nombre de prestations LCA par an et par patient
- Montant total de prestations LCA par an et par patient
- Nombres de factures par an et par patient
- Nb quantité par an et par patient

### Identify prestations that are clearly CAM and the ones that are quite ambiguous

### Define a complementary medicine status

Some claims are clearly complementary medicine as usually understood (a therapist with known practices, a code related to it, etc.) But there are also claimed billed by Doctors, Dentists, Fitness centers. These should be flagged. 

In [ ]:
df_prestation_lca['medcomp_status'] = np.nan

In [ ]:
df_prestation_lca.loc[~df_prestation_lca.CODES_THERAPIES.isnull(), 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.CATEGORIE_DISPENSATEUR == 'Pharmacies', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Medecine douce', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Medicament medecine douce', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Medecine douce medecins', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Acupuncture et homeopathie', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Osteopathie et etiopathie', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Sophrologie', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Soins balneaires', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Naturopathie', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Soins dietetiques', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Electroacupuncture', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == 'Medecines douces massage', 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == "Soins balneaires a l'etranger", 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == "Soins chiropratiques", 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == "Soins orthoptiques", 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == "Prestations Yoga", 'medcomp_status'] = 'Clear'
df_prestation_lca.loc[df_prestation_lca.TXGENREFRAISLGFR == "Soins podologiques", 'medcomp_status'] = 'Clear'

In [ ]:
df_prestation_lca[df_prestation_lca.medcomp_status != 'Clear'].TXGENREFRAISLGFR.value_counts()

### Features

In [ ]:
n_prestation_lca_by_patient_by_month = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT'], observed = True).size()
sum_prestation_lca_by_patient_by_month = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT'], observed = True).PRESTATIONS_BRUTES.sum()
n_bill_prestation_lca_by_patient_by_month = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT'], observed = True).NBRE_FACTURES.sum()
n_quantity_prestation_lca_by_patient_by_month = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT'], observed = True).NBQUANTITE.sum()

In [ ]:
n_prestation_lca_by_patient_by_year = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT'], observed = True).size()
sum_prestation_lca_by_patient_by_year = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT'], observed = True).PRESTATIONS_BRUTES.sum()
n_bill_prestation_lca_by_patient_by_year = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT'], observed = True).NBRE_FACTURES.sum()
n_quantity_prestation_lca_by_patient_by_year = df_prestation_lca[df_prestation_lca.medcomp_status == 'Clear'].groupby(['uuid','ANNEE_TRAITEMENT'], observed = True).NBQUANTITE.sum()

In [ ]:
sum_prestation_lca_by_patient_by_year.sum()/1000000

In [ ]:
sum_prestation_lca_by_patient_by_month = pd.DataFrame(sum_prestation_lca_by_patient_by_month)
sum_prestation_lca_by_patient_by_month = sum_prestation_lca_by_patient_by_month.reset_index()

In [ ]:
n_quantity_prestation_lca_by_patient_by_month = pd.DataFrame(n_quantity_prestation_lca_by_patient_by_month)
n_quantity_prestation_lca_by_patient_by_month = n_quantity_prestation_lca_by_patient_by_month.reset_index()

In [ ]:
# Step 1: Calculate the cumulative sum within each year
sum_prestation_lca_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] = sum_prestation_lca_by_patient_by_month.sort_values(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT']).groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].cumsum()
month_reached_500_lca = sum_prestation_lca_by_patient_by_month[sum_prestation_lca_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 500].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()

month_reached_1000_lca = sum_prestation_lca_by_patient_by_month[sum_prestation_lca_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 1000].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()
month_reached_2500_lca = sum_prestation_lca_by_patient_by_month[sum_prestation_lca_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 2500].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()

month_reached_5000_lca = sum_prestation_lca_by_patient_by_month[sum_prestation_lca_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 5000].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()
month_reached_10000_lca = sum_prestation_lca_by_patient_by_month[sum_prestation_lca_by_patient_by_month['PRESTATIONS_BRUTES_CUMSUM'] >= 10000].groupby(['uuid','ANNEE_TRAITEMENT'])['MOIS_TRAITEMENT'].min()

In [ ]:
# Highest month
max_monthly_lca_cost = sum_prestation_lca_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].max()

In [ ]:
# Monthly average
mean_monthly_lca_cost = sum_prestation_lca_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].mean()
# Frequency average

mean_monthly_lca_freq = n_quantity_prestation_lca_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT']).size()/12

In [ ]:
# Mean three last months
last_3months_total_lca_cost = sum_prestation_lca_by_patient_by_month[sum_prestation_lca_by_patient_by_month.MOIS_TRAITEMENT.isin([10,11,12])].groupby(['uuid','ANNEE_TRAITEMENT'])['PRESTATIONS_BRUTES'].sum()

In [ ]:
# Number of months, weeks using LCA
n_month_lca_by_patient = sum_prestation_lca_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT']).size()

In [ ]:
# df_prestation_lca_outcomes['treatmentdate'] = pd.to_datetime(df_prestation_lca_outcomes['ANNEE_TRAITEMENT'].astype('string') + '-' + df_prestation_lca_outcomes['MOIS_TRAITEMENT'].astype('string'))
# df_prestation_lca_outcomes['treatmentmonth'] = df_prestation_lca_outcomes['treatmentdate'].dt.strftime('%Y-%m')

## Cleaning of LCA therapy types
##### 1. Import the simplified ontology for LCA therapies previously created (details in Notebook Preparation GM - LCA ontology simplification)

In [ ]:
filtered_therap_clean = pd.read_csv('../Data/processed/20230223_Therapies_ontology.csv',encoding='ISO-8859-1')

Les données sur les codes ASCA sont disponibles [ici](http://asca.ch/therapies.aspx?all=true)

In [ ]:
# filtered_therap_clean['Code'] = filtered_therap_clean['Code'].str.replace(' ','')
filtered_therap_clean['Code'] = filtered_therap_clean['Code'].str.strip()

In [ ]:
dict_ontology_lca = filtered_therap_clean.set_index('Code')['therapie_lvl2'].to_dict()
dict_ontology_lca_by_discipline = filtered_therap_clean.set_index('Code')['Methode'].to_dict()

In [ ]:
df_prestation_lca.CATEGORIE_DISPENSATEUR.value_counts()

In [ ]:
df_prestation_lca.TXGENREFRAISLGFR.value_counts(normalize = True).mul(100).round(2)

In [ ]:
df_prestation_lca[df_prestation_lca.medcomp_status.isnull()].TXGENREFRAISLGFR.value_counts()

In [ ]:
df_dispensateurs_lca = df_prestation_lca[['ID_DISPENSATEUR','CODES_THERAPIES','THERAPIES','TXGENREFRAISLGFR','CATEGORIE_DISPENSATEUR']].drop_duplicates()

In [ ]:
df_dispensateurs_lca['CODES_THERAPIES'] = df_dispensateurs_lca['CODES_THERAPIES'].str.split(',')
df_dispensateurs_lca['THERAPIES'] = df_dispensateurs_lca['THERAPIES'].str.split(',')

def strip_if_list(item):
    if isinstance(item, list):
        return [str_elem.strip() for str_elem in item]
    elif isinstance(item, str):
        return item.strip()
    return item

# Apply the function to the DataFrame column
df_dispensateurs_lca['CODES_THERAPIES'] = df_dispensateurs_lca['CODES_THERAPIES'].apply(strip_if_list)
df_dispensateurs_lca['THERAPIES'] = df_dispensateurs_lca['THERAPIES'].apply(strip_if_list)

In [ ]:
def simplify_ontology(x, dict_mapping):
    try:
        x = [i.strip() for i in x]
        y = list(dict_mapping[i] if i in dict_mapping.keys() else i for i in x)
        return y
    except:
        return np.nan

In [ ]:
# Function to process each row - 
## For each row, any string containing the substring APTN in CODES_THERAPIES is removed from CODES_THERAPIES
## For each row, any string containing the subtring APTN in THERAPIES is added in CODES_THERAPIES
def process_row(row):
    if isinstance(row['CODES_THERAPIES'], list) and isinstance(row['THERAPIES'], list):
        # Remove elements with 'APTN' from CODES_THERAPIES
        row['CODES_THERAPIES'] = [code for code in row['CODES_THERAPIES'] if 'APTN' not in code]
        
        # Extract codes with 'APTN' from THERAPIES and add to CODES_THERAPIES
        aptn_codes = [therapy for therapy in row['THERAPIES'] if 'APTN' in therapy]
        row['CODES_THERAPIES'].extend(aptn_codes)
        
    return row

In [ ]:
# Apply the function row-wise
df_dispensateurs_lca = df_dispensateurs_lca.apply(process_row, axis=1)

In [ ]:
df_dispensateurs_lca['THERAPIES_SIMPLIFIED'] = df_dispensateurs_lca.apply(lambda x : simplify_ontology(x['CODES_THERAPIES'], dict_ontology_lca), axis = 1)
df_dispensateurs_lca['DISCIPLINES_SIMPLIFIED'] = df_dispensateurs_lca.apply(lambda x : simplify_ontology(x['CODES_THERAPIES'], dict_ontology_lca_by_discipline), axis = 1)

In [ ]:
unique_codes_lca = df_dispensateurs_lca['CODES_THERAPIES'].explode().unique()

In [ ]:
# df_dispensateurs_lca['THERAPIES'] = df_dispensateurs_lca['THERAPIES'].astype(str)

In [ ]:
unique_therapies_lca = df_dispensateurs_lca[df_dispensateurs_lca.THERAPIES.isnull()==False]['THERAPIES'].explode().unique().tolist()

In [ ]:
codes_lca_manquants = []
for code_lca in unique_codes_lca:
    try:
        print(dict_ontology_lca[code_lca.strip()])
    except:
        print('Code manquant', code_lca)
        codes_lca_manquants.append(code_lca)

In [ ]:
len(codes_lca_manquants)

In [ ]:
df_dispensateurs_lca_exploded = df_dispensateurs_lca.explode(['CODES_THERAPIES','THERAPIES_SIMPLIFIED','DISCIPLINES_SIMPLIFIED'])

In [ ]:
# TOP 50 most practiced therapy types 
df_dispensateurs_lca_exploded.groupby('THERAPIES_SIMPLIFIED').ID_DISPENSATEUR.nunique().sort_values().tail(50)

In [ ]:
# Save the exploded dispensers DataFrame to a parquet file
df_dispensateurs_lca_exploded.to_parquet(
    data_folder/'processed'/'df_dispensateur_lca_exploded.parquet.gzip', compression='gzip'
)

In [ ]:
# Create a function to handle float inputs and convert non-floats into sets
def set_or_nan(x):
    if isinstance(x, float):
        return float('nan')
    else:
        return set(x)

# Apply the function to the 'THERAPIES_SIMPLIFIED' column
df_dispensateurs_lca['THERAPIES_SIMPLIFIED_SET'] = df_dispensateurs_lca['THERAPIES_SIMPLIFIED'].apply(set_or_nan)
df_dispensateurs_lca['DISCIPLINES_SIMPLIFIED_SET'] = df_dispensateurs_lca['DISCIPLINES_SIMPLIFIED'].apply(set_or_nan)

# Group by 'ID_DISPENSATEUR', aggregate unique therapies for each dispenser
df_dispensateurs_lca_nodupli_a = df_dispensateurs_lca.groupby('ID_DISPENSATEUR').agg({
    'THERAPIES_SIMPLIFIED_SET': lambda x: set().union(*[y for y in x if isinstance(y, set)])
})
df_dispensateurs_lca_nodupli_b = df_dispensateurs_lca.groupby('ID_DISPENSATEUR').agg({
    'DISCIPLINES_SIMPLIFIED_SET': lambda x: set().union(*[y for y in x if isinstance(y, set)])
})

df_dispensateurs_lca_nodupli = pd.merge(df_dispensateurs_lca_nodupli_a, df_dispensateurs_lca_nodupli_b, left_index = True, right_index = True)
# Count the number of unique therapies for each dispenser
df_dispensateurs_lca_nodupli['n_therapies'] = df_dispensateurs_lca_nodupli['THERAPIES_SIMPLIFIED_SET'].apply(len)
df_dispensateurs_lca_nodupli['n_disciplines'] = df_dispensateurs_lca_nodupli['DISCIPLINES_SIMPLIFIED_SET'].apply(len)

# Reset the DataFrame index
df_dispensateurs_lca_nodupli = df_dispensateurs_lca_nodupli.reset_index()

# Explode the 'THERAPIES_SIMPLIFIED_SET' column into multiple rows, duplicating the values of the other columns
df_dispensateurs_lca_nodupli_exploded = df_dispensateurs_lca_nodupli.explode('THERAPIES_SIMPLIFIED_SET')

# Create a dummy column for pivoting purposes
df_dispensateurs_lca_nodupli_exploded['dummy'] = 1

# Pivot the DataFrame to get a one-hot encoded matrix for the therapies, replacing NaNs with 0
df_dispensateurs_lca_nodupli_pivot = df_dispensateurs_lca_nodupli_exploded.explode('THERAPIES_SIMPLIFIED_SET').pivot(
    index='ID_DISPENSATEUR', columns='THERAPIES_SIMPLIFIED_SET', values='dummy'
).fillna(0)

# Sort dispensers by the number of therapies they offer
df_dispensateurs_lca_nodupli.sort_values('n_therapies')

In [ ]:
df_dispensateurs_lca_nodupli.groupby('n_disciplines').size()

In [ ]:
df_dispensateurs_lca_nodupli[df_dispensateurs_lca_nodupli.n_disciplines ==13]

In [ ]:
n_1_thera_disp = df_dispensateurs_lca_nodupli[df_dispensateurs_lca_nodupli.n_therapies == 1].ID_DISPENSATEUR

In [ ]:
df_dispensateurs_lca_nodupli.groupby('n_disciplines').size()

In [ ]:
df_dispensateurs_lca_nodupli[df_dispensateurs_lca_nodupli.n_disciplines == 0]

In [ ]:
# Save the exploded dispensers DataFrame to a parquet file
df_dispensateurs_lca_nodupli.to_parquet(
    data_folder/'processed'/'df_dispensateurs_lca_nodupli.parquet.gzip', compression='gzip'
)

In [ ]:
# if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_souscat_disp.parquet.gzip'):

#     df_amount_by_souscat_disp = pd.DataFrame(df_prestation_aos.groupby(['uuid','ANNEE_TRAITEMENT','SOUS_CATEGORIE_DISPENSATEUR']).PRESTATIONS_BRUTES.sum()).reset_index()
#     df_amount_by_souscat_disp = df_amount_by_souscat_disp.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'SOUS_CATEGORIE_DISPENSATEUR', values = 'PRESTATIONS_BRUTES').reset_index()
#     df_amount_by_souscat_disp.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_souscat_disp.parquet.gzip', compression = 'gzip')
# else:
#     df_amount_by_souscat_disp = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_souscat_disp.parquet.gzip')

#### After cleaning, filtering and removing duplicates in the THERAPIES, we add the column to prestation_lca. 

**TO DO** : 

- There is a big chance that some therapists have changing CODES and THERAPIES over years, this is not taken into account atm. Something that could be done in the future.
- Check if there is a way to disentangle which bill is associated to which THERAPY based on the amount. To know which price is associated to which therapy, we can use the therapists having only one specialty.
    - We don't have the number of sessions that were agregated in the monthly agregation -> we have amounts that go up to > 3000CHF ! We thus have another step to find the "unique" sessions and their price. We can then find the number of sessions agregated. Cool cool

In [ ]:
# Merging df_prestation_lca with df_dispensateurs_lca_nodupli on 'ID_DISPENSATEUR'
# keeping only the relevant columns ('ID_DISPENSATEUR', 'THERAPIES_SIMPLIFIED_SET', 'n_therapies') from the latter.
# 'how = left' ensures all rows from df_prestation_lca are retained and filled with NaN for missing matches.
df_prestation_lca = pd.merge(df_prestation_lca, 
                             df_dispensateurs_lca_nodupli[['ID_DISPENSATEUR','THERAPIES_SIMPLIFIED_SET','n_therapies','DISCIPLINES_SIMPLIFIED_SET','n_disciplines']], 
                             on = 'ID_DISPENSATEUR', 
                             how = 'left')

In [ ]:
conditions = [
    (df_prestation_lca.CATEGORIE_DISPENSATEUR == 'Pharmacies', {'Methodes prescriptives'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Medicament medecine douce', {'Methodes prescriptives'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Acupuncture et homeopathie', {'Methodes orientales','Methodes prescriptives'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Osteopathie et etiopathie', {'Methodes energetiques manuelles'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Sophrologie', {'Methodes psychologiques complementaires'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Soins balneaires', {'Methodes hydrotherapeutiques'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Naturopathie', {'Methodes orientales'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Soins dietetiques', {'Methodes prescriptives'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Electroacupuncture', {'Methodes orientales'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Medecines douces massage', {'Methodes de massage'}),
    (df_prestation_lca.TXGENREFRAISLGFR == "Soins balneaires a l'etranger", {'Methodes psychologiques complementaires'}),
    (df_prestation_lca.TXGENREFRAISLGFR == 'Soins chiropratiques', {'Methodes energetiques manuelles'})
]

for condition, value_set in conditions:
    df_prestation_lca.loc[condition, 'DISCIPLINES_SIMPLIFIED_SET'] = df_prestation_lca.loc[condition, 'DISCIPLINES_SIMPLIFIED_SET'].apply(lambda x: value_set if pd.isna(x) else x)

In [ ]:
df_prestation_lca['n_disciplines'] = df_prestation_lca['DISCIPLINES_SIMPLIFIED_SET'].apply(len)

In [ ]:
# Calculate the percentage of prestations done by therapists with 1, 2, and less than 5 specialties respectively
pct_single_specialty = (df_prestation_lca[df_prestation_lca.n_therapies == 1].shape[0] / df_prestation_lca.shape[0]) * 100
pct_two_specialties = (df_prestation_lca[df_prestation_lca.n_therapies <= 2].shape[0] / df_prestation_lca.shape[0]) * 100
pct_less_five_specialties = (df_prestation_lca[df_prestation_lca.n_therapies < 5].shape[0] / df_prestation_lca.shape[0]) * 100

# Use formatted string literals (f-strings) for more readable print statements
print(f'About {pct_single_specialty:.1f}% of LCA prestations are done by a therapist having 1 specialty')
print(f'About {pct_two_specialties:.1f}% of LCA prestations are done by a therapist having 1 or 2 specialties')
print(f'About {pct_less_five_specialties:.1f}% of LCA prestations are done by a therapist having less than 5 specialties')

In [ ]:
# Calculate the percentage of prestations done by therapists with 1, 2, and less than 5 specialties respectively
pct_zero_discipline = (df_prestation_lca[df_prestation_lca.n_disciplines == 0].shape[0] / df_prestation_lca.shape[0]) * 100
pct_single_discipline = (df_prestation_lca[df_prestation_lca.n_disciplines == 1].shape[0] / df_prestation_lca.shape[0]) * 100
pct_two_disciplines = (df_prestation_lca[df_prestation_lca.n_disciplines <= 2].shape[0] / df_prestation_lca.shape[0]) * 100
pct_less_five_disciplines = (df_prestation_lca[df_prestation_lca.n_disciplines < 5].shape[0] / df_prestation_lca.shape[0]) * 100
pct_less_ten_disciplines = (df_prestation_lca[df_prestation_lca.n_disciplines < 10].shape[0] / df_prestation_lca.shape[0]) * 100

# Use formatted string literals (f-strings) for more readable print statements
print(f'About {pct_zero_discipline:.1f}% of LCA prestations are done by a therapist without a discipline')
print(f'About {pct_single_discipline:.1f}% of LCA prestations are done by a therapist having 1 discipline')
print(f'About {pct_two_disciplines:.1f}% of LCA prestations are done by a therapist having 1 or 2 disciplines')
print(f'About {pct_less_five_disciplines:.1f}% of LCA prestations are done by a therapist having less than 5 disciplines')
print(f'About {pct_less_ten_disciplines:.1f}% of LCA prestations are done by a therapist having less than 10 disciplines')

In [ ]:
df_prestation_lca.groupby('n_disciplines').size().sort_values()

That means that we have a big third that is fine, another big third that is potentially manageable...the rest...will be quite hard to manage !

In [ ]:
df_prestation_lca[df_prestation_lca.n_therapies > 2].uuid.nunique()

#### Number by type of therapy code LCA

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_n_by_lca_therapy.parquet.gzip'):
    df_n_by_lca_therapy = df_prestation_lca.explode('THERAPIES_SIMPLIFIED_SET')
    df_n_by_lca_therapy = pd.DataFrame(df_n_by_lca_therapy.groupby(['uuid','ANNEE_TRAITEMENT','THERAPIES_SIMPLIFIED_SET']).size()).reset_index()
    df_n_by_lca_therapy = df_n_by_lca_therapy.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'THERAPIES_SIMPLIFIED_SET', values = 0).reset_index()
    df_n_by_lca_therapy.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_n_by_lca_therapy.parquet.gzip', compression = 'gzip')
else:
    df_n_by_lca_therapy = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_n_by_lca_therapy.parquet.gzip')

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_lca_therapy.parquet.gzip'):
    df_amount_by_lca_therapy = df_prestation_lca.explode('THERAPIES_SIMPLIFIED_SET')
    df_amount_by_lca_therapy = pd.DataFrame(df_amount_by_lca_therapy.groupby(['uuid','ANNEE_TRAITEMENT','THERAPIES_SIMPLIFIED_SET']).PRESTATIONS_BRUTES.sum()).reset_index()
    df_amount_by_lca_therapy = df_amount_by_lca_therapy.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'THERAPIES_SIMPLIFIED_SET', values = 'PRESTATIONS_BRUTES').reset_index()
    df_amount_by_lca_therapy.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_lca_therapy.parquet.gzip', compression = 'gzip')
else:
    df_amount_by_lca_therapy = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_lca_therapy.parquet.gzip')

#### Number and amount by LCA discipline

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_n_by_lca_discipline.parquet.gzip'):
    df_n_by_lca_discipline = df_prestation_lca.explode('DISCIPLINES_SIMPLIFIED_SET')
    df_n_by_lca_discipline = pd.DataFrame(df_n_by_lca_discipline.groupby(['uuid','ANNEE_TRAITEMENT','DISCIPLINES_SIMPLIFIED_SET']).size()).reset_index()
    df_n_by_lca_discipline = df_n_by_lca_discipline.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'DISCIPLINES_SIMPLIFIED_SET', values = 0).reset_index()
    df_n_by_lca_discipline.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_n_by_lca_discipline.parquet.gzip', compression = 'gzip')
else:
    df_n_by_lca_discipline = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_n_by_lca_discipline.parquet.gzip')

In [ ]:
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_lca_discipline.parquet.gzip'):
    df_amount_by_lca_discipline = df_prestation_lca.explode('DISCIPLINES_SIMPLIFIED_SET')
    df_amount_by_lca_discipline = pd.DataFrame(df_amount_by_lca_discipline.groupby(['uuid','ANNEE_TRAITEMENT','DISCIPLINES_SIMPLIFIED_SET']).PRESTATIONS_BRUTES.sum()).reset_index()
    df_amount_by_lca_discipline = df_amount_by_lca_discipline.pivot(index = ['uuid','ANNEE_TRAITEMENT'], columns = 'DISCIPLINES_SIMPLIFIED_SET', values = 'PRESTATIONS_BRUTES').reset_index()
    df_amount_by_lca_discipline.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_lca_discipline.parquet.gzip', compression = 'gzip')
else:
    df_amount_by_lca_discipline = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_lca_discipline.parquet.gzip')

In [ ]:
n_dispensateur_lca = df_prestation_lca.groupby(['uuid','ANNEE_TRAITEMENT']).ID_DISPENSATEUR.apply(set).apply(lambda x: len(x) if isinstance(x, set) else 0)

## Final LCA features 

In [ ]:
df_prestation_lca_outcomes_by_year = pd.concat([n_prestation_lca_by_patient_by_year, sum_prestation_lca_by_patient_by_year, n_bill_prestation_lca_by_patient_by_year, n_quantity_prestation_lca_by_patient_by_year,month_reached_500_lca, month_reached_1000_lca, month_reached_2500_lca, month_reached_5000_lca, month_reached_10000_lca, max_monthly_lca_cost, mean_monthly_lca_cost, mean_monthly_lca_freq, last_3months_total_lca_cost, n_month_lca_by_patient, n_dispensateur_lca], axis = 1).reset_index().rename(columns = {0:'NBROWS'})
# df_prestation_lca_outcomes_by_year = df_prestation_lca_outcomes.groupby(['uuid','ANNEE_TRAITEMENT'], observed = True)[['NBROWS','PRESTATIONS_BRUTES','NBRE_FACTURES','NBQUANTITE']].sum().reset_index()

In [ ]:
df_prestation_lca_outcomes_by_year.columns = ['uuid', 'ANNEE_TRAITEMENT', 'NBROWS', 'PRESTATIONS_BRUTES',
       'NBRE_FACTURES', 'NBQUANTITE','month_reached_500_lca','month_reached_1000_lca','month_reached_2500_lca','month_reached_5000_lca','month_reached_10000_lca','max_monthly_lca','mean_monthly_lca','mean_monthly_nbquantite_lca','last3month_lca','n_month_lca_by_patient', 'n_dispensateur_lca']

In [ ]:
df_prestation_lca_outcomes_by_year = pd.merge(df_prestation_lca_outcomes_by_year, df_n_by_lca_discipline.fillna(0), on = ['uuid','ANNEE_TRAITEMENT'], how = 'outer').merge(df_amount_by_lca_discipline.fillna(0), on = ['uuid','ANNEE_TRAITEMENT'], how = 'outer', suffixes = ('_n','_amount'))

In [ ]:
df_prestation_lca_outcomes_by_year.PRESTATIONS_BRUTES.sum()/1000000

Still all good at this stage

In [ ]:
del n_prestation_lca_by_patient_by_year, n_quantity_prestation_lca_by_patient_by_year, df_n_by_lca_discipline, df_amount_by_lca_discipline, month_reached_500_lca, month_reached_1000_lca, month_reached_2500_lca, month_reached_5000_lca, month_reached_10000_lca, max_monthly_lca_cost, mean_monthly_lca_cost, mean_monthly_lca_freq, last_3months_total_lca_cost

## Save dataset of prestations LCA

In [ ]:
df_prestation_lca.to_parquet(data_folder/'processed'/'df_prestation_lca_processed.parquet_gzip', compression='gzip')

### Creating a DataFrame of Therapy-Price Pairs

In [ ]:
## Selecting the dispensateurs having only 1 THERAPY type
df_prestation_lca_1spe = df_prestation_lca[df_prestation_lca.n_therapies == 1].explode('THERAPIES_SIMPLIFIED_SET')

##### Check the most expensive therapies

In [ ]:
df_prestation_lca_1spe[['ID_LCA','MOIS_TRAITEMENT','ANNEE_TRAITEMENT','CODES_THERAPIES','TXGENREFRAISLGFR','PRESTATIONS_BRUTES','NBRE_FACTURES','NBQUANTITE','THERAPIES_SIMPLIFIED_SET']].sort_values(['PRESTATIONS_BRUTES']).tail(20)

In [ ]:
fig, ax = plt.subplots(figsize = (10,30))
sns.barplot(data=df_prestation_lca_1spe, x="PRESTATIONS_BRUTES",y = 'THERAPIES_SIMPLIFIED_SET',dodge=False, hue="THERAPIES_SIMPLIFIED_SET", ax = ax)

We note that actually, this way, we don't find the most expensive therapies but the highest monthly total amounts. We need to filter out all the amounts that are obviously the result of the monthly agregation of multiple therapy sessions. Then, we may be able to find a price per type of therapy.

Let's select only the `prestations` which an amount < 200CHF. 

In [ ]:
df_prestation_lca_1spe_1session = df_prestation_lca_1spe[df_prestation_lca_1spe.PRESTATIONS_BRUTES < 150]

In [ ]:
ordered_labels = df_prestation_lca_1spe_1session.groupby('THERAPIES_SIMPLIFIED_SET')['PRESTATIONS_BRUTES'].median().sort_values().index

In [ ]:
fig, ax = plt.subplots(figsize = (8,15))
# Plot the orbital period with horizontal boxes
sns.boxplot(data=df_prestation_lca_1spe_1session, x="PRESTATIONS_BRUTES",y = 'THERAPIES_SIMPLIFIED_SET', order = ordered_labels, width=.6, palette="vlag", ax = ax)
plt.savefig(result_folder/"EDA"/'Therapy-Price pairs'/'Box plot - Montants par type de thérapie.pdf', bbox_inches = 'tight')

#### Filtering out everything that is not a session (taxes and medicines)

In [ ]:
df_prestation_lca_1spe_1session_notax = df_prestation_lca_1spe_1session[~df_prestation_lca_1spe_1session.TXGENREFRAISLGFR.isin(['Medicaments hors liste','Medicament medecine douce','Taxe pour livraison'])]

In [ ]:
ordered_labels = df_prestation_lca_1spe_1session_notax.groupby('THERAPIES_SIMPLIFIED_SET')['PRESTATIONS_BRUTES'].median().sort_values().index

In [ ]:
fig, ax = plt.subplots(figsize = (8,15))
# Plot the orbital period with horizontal boxes
sns.boxplot(data=df_prestation_lca_1spe_1session_notax, x="PRESTATIONS_BRUTES",y = 'THERAPIES_SIMPLIFIED_SET', order = ordered_labels, width=.6, palette="vlag", ax = ax)
plt.savefig(result_folder/"EDA"/'Therapy-Price pairs'/'Box plot - Montants par type de thérapie - Taxes et médicaments exclus.pdf', bbox_inches = 'tight')

In [ ]:
df_ostheo = df_prestation_lca_1spe_1session_notax[df_prestation_lca_1spe_1session_notax['THERAPIES_SIMPLIFIED_SET'] == 'Osteopathie']
df_massage = df_prestation_lca_1spe_1session_notax[df_prestation_lca_1spe_1session_notax['THERAPIES_SIMPLIFIED_SET'] == 'Massage classique']

In [ ]:
sns.violinplot(data = df_ostheo, x = 'PRESTATIONS_BRUTES')

In [ ]:
sns.violinplot(data = df_massage[df_massage.ID_LCA.isin(df_massage['ID_LCA'].sample(1))], x = 'PRESTATIONS_BRUTES')

With a global approach like this, we see that it will be very hard to create a therapy:price dataframe. Prices may vary by region, by therapist and over the study period. Probably a better strategy is to check the different prices that each therapist has practiced over the 5 years. If there are a lot, maybe we can have something like this:

|ID| YEAR |THERAPY|PRICE|
|---|---|---|---|
|123432| 2020| Massage|96|
|123432| 2021| Massage|102|
|123433| 2021| Osteopathy|120|




In [ ]:
df_prestation_lca_1spe_1session_notax['PRESTATIONS_BRUTES_rounded'] = df_prestation_lca_1spe_1session_notax['PRESTATIONS_BRUTES'].round(0)

In [ ]:
pd.DataFrame(df_prestation_lca_1spe_1session_notax.groupby(['ID_DISPENSATEUR','ANNEE_TRAITEMENT','THERAPIES_SIMPLIFIED_SET'])['PRESTATIONS_BRUTES_rounded'].unique()).reset_index()

**Conclusions** : This is way messier than we could have hoped for...It will be almost impossible to infer what therapy has been used during a session for any session done at a therapist having more than 2 specialties. Even then, it may be challenging since the prices are all over the place !

**TO DO**: Cry ... although ... It seems that the pricing is actually quite uniform ! This should be revisited. For example, the price of a visit at the osteo is normally 120 CHF. There is apparently a standardized tarification that is explained here : https://terap.ch/fr/blog/tarif-590 



### Check if prices vary by canton

In [ ]:
df_prestation_lca_1spe_1session_notax = pd.merge(df_prestation_lca_1spe_1session_notax, df_couverture_aos[['uuid','CDCANTON_POST']].drop_duplicates(subset='uuid'), on = ['uuid'], how = 'left')

In [ ]:
df_ostheo = df_prestation_lca_1spe_1session_notax[df_prestation_lca_1spe_1session_notax['THERAPIES_SIMPLIFIED_SET'] == 'Osteopathie']
df_massage = df_prestation_lca_1spe_1session_notax[df_prestation_lca_1spe_1session_notax['THERAPIES_SIMPLIFIED_SET'] == 'Massage classique']

In [ ]:
ordered_labels = df_ostheo.groupby('CDCANTON_POST')['PRESTATIONS_BRUTES'].median().sort_values().index
fig, ax = plt.subplots(figsize = (6,10))

sns.boxplot(data = df_ostheo, x = 'PRESTATIONS_BRUTES',order = ordered_labels,y = 'CDCANTON_POST', width=.6, palette="vlag", ax = ax)

In [ ]:
df_ostheo[df_ostheo.CDCANTON_POST == 'VS']['PRESTATIONS_BRUTES'].plot.hist(bins = 50)

In [ ]:
ordered_labels = df_ostheo.groupby('CDCANTON_POST')['PRESTATIONS_BRUTES'].median().sort_values().index
fig, ax = plt.subplots(figsize = (6,10))

sns.boxplot(data = df_massage, x = 'PRESTATIONS_BRUTES',order = ordered_labels,y = 'CDCANTON_POST', width=.6, palette="vlag", ax = ax)

### Combine LCA and AOS

In [ ]:
# Renaming columns for clarity
df_prestation_lca_outcomes_by_year = df_prestation_lca_outcomes_by_year.rename(columns = {
    'NBROWS': "NBROWS_LCA",
    'PRESTATIONS_BRUTES': "PRESTATIONS_BRUTES_LCA",
    'NBRE_FACTURES': "NBRE_FACTURES_LCA",
    'NBQUANTITE': "NBQUANTITE_LCA"})

df_prestation_aos_outcomes_by_year = df_prestation_aos_outcomes_by_year.rename(columns = {
    'NBROWS': "NBROWS_AOS",
    'PRESTATIONS_BRUTES': "PRESTATIONS_BRUTES_AOS",
    "PRESTATIONS_NETTES": "PRESTATIONS_NETTES_AOS",
    'NBRE_FACTURES': "NBRE_FACTURES_AOS",
    'NBQUANTITE': "NBQUANTITE_AOS"})

# Combining the two dataframes by setting 'uuid' and 'ANNEE_TRAITEMENT' as index and concatenating along columns
df_outcomes_prestation = pd.concat(
    [df_prestation_lca_outcomes_by_year.set_index(['uuid','ANNEE_TRAITEMENT']), 
     df_prestation_aos_outcomes_by_year.set_index(['uuid','ANNEE_TRAITEMENT'])], 
    axis = 1).reset_index()

# Filling NaN values with 0.0
# Get columns that don't contain 'month_'
cols_to_fill = df_outcomes_prestation.filter(regex='^(?!.*month_).*$').columns

# Fill NaN values in these columns
df_outcomes_prestation[cols_to_fill] = df_outcomes_prestation[cols_to_fill].fillna(0.0)

# Creating new columns for total prestations and total number of invoices
df_outcomes_prestation['PRESTATIONS_TOTAL'] = df_outcomes_prestation[['PRESTATIONS_BRUTES_AOS', 'PRESTATIONS_BRUTES_LCA']].sum(axis = 1)
df_outcomes_prestation['NBRE_FACTURES_TOTAL'] = df_outcomes_prestation[['NBRE_FACTURES_AOS', 'NBRE_FACTURES_LCA']].sum(axis = 1)

# Merging with df_gps_exploded_excl_gp on ['ANNEE_TRAITEMENT','uuid'] and ["NOANNEE", "uuid"] respectively
df_outcomes_prestation = df_outcomes_prestation.merge(
    df_gps_exploded_excl_gp, 
    left_on=['ANNEE_TRAITEMENT','uuid'], 
    right_on=["NOANNEE", "uuid"], 
    how="left")

# Mapping the 'gp' column using dict_labels_gps
df_outcomes_prestation['gp'] = df_outcomes_prestation['gp'].map(dict_labels_gps)

# Filtering out rows where 'gp' contains 'No usage'
df_outcomes_prestation = df_outcomes_prestation[~df_outcomes_prestation.gp.str.contains('No usage', na = False)]


In [ ]:
df_outcomes_prestation.gp.unique()

In [ ]:
df_outcomes_prestation[df_outcomes_prestation.gp.isnull()]

In [ ]:
df_outcomes_prestation.PRESTATIONS_BRUTES_LCA.sum()

In [ ]:
df_outcomes_prestation.PRESTATIONS_BRUTES_AOS.sum()

**Still all good for the total sum**

In [ ]:
df_outcomes_prestation.groupby('gp', observed = True).PRESTATIONS_TOTAL.mean().plot.bar()

In [ ]:
# Merge 'df_outcomes_prestation' with 'df_amount_by_souscat_disp' on keys ['uuid','ANNEE_TRAITEMENT']
# Fill NaN values with 0 after the merge
df_outcomes_prestation_temp = pd.merge(
    df_outcomes_prestation, 
    df_amount_by_souscat_disp, 
    on=['uuid','ANNEE_TRAITEMENT'], 
    how='left'
)

# Merge the above merged dataframe with 'df_amount_by_type_prestation' on keys ['uuid','ANNEE_TRAITEMENT']
# Fill NaN values with 0 after the merge
df_outcomes_prestation_temp = pd.merge(
    df_outcomes_prestation_temp, 
    df_amount_by_type_prestation.fillna(0), 
    on=['uuid','ANNEE_TRAITEMENT'], 
    how='outer'
)

# Merge the above merged dataframe with 'df_amount_by_sinistre' on keys ['uuid','ANNEE_TRAITEMENT']
# Fill NaN values with 0 after the merge
df_outcomes_prestation_temp = pd.merge(
    df_outcomes_prestation_temp, 
    df_amount_by_sinistre.fillna(0), 
    on=['uuid','ANNEE_TRAITEMENT'], 
    how='outer'
)

# Merge the above merged dataframe with 'df_outcomes_hosp' on keys ['uuid','ANNEE_TRAITEMENT'] and ['uuid','year'] respectively
# Fill NaN values with 0 after the merge
df_outcomes_prestation_temp = pd.merge(
    df_outcomes_prestation_temp, 
    df_outcomes_hosp,
    left_on=['uuid','ANNEE_TRAITEMENT'], 
    right_on=['uuid','year'], 
    how='outer'
)

# Merge the above merged dataframe with 'df_n_by_lca_therapy' on keys ['uuid','ANNEE_TRAITEMENT'] and ['uuid','year'] respectively
# Fill NaN values with 0 after the merge
df_outcomes_prestation_temp = pd.merge(
    df_outcomes_prestation_temp, 
    df_n_by_lca_therapy.fillna(0),
    left_on=['uuid','ANNEE_TRAITEMENT'], 
    right_on=['uuid','ANNEE_TRAITEMENT'], 
    how='outer'
)


In [ ]:
# Renaming columns 'A', 'B' and 'M' to 'PRESTATIONS_ACCIDENT', 'PRESTATIONS_BIRTH' and 'PRESTATIONS_DISEASE'
# This is done for clarity and ease of understanding during subsequent analysis
df_outcomes_prestation_temp = df_outcomes_prestation_temp.rename(
    columns={
        'A':'PRESTATIONS_ACCIDENT',
        'B':'PRESTATIONS_BIRTH', 
        'M':'PRESTATIONS_DISEASE'
    }
)

In [ ]:
df_outcomes_prestation_temp['PRESTATIONS_BRUTES_AOS'].sum()

In [ ]:
df_outcomes_prestation_temp['PRESTATIONS_BRUTES_LCA'].sum()

## Prescription data

### Total number of prescriptions, total amount by year

In [ ]:
nb_and_amount_prescriptions_by_year = optimize_memory_df(df_drug_aos.groupby(['uuid','ANNEE_TRAITEMENT'], observed = True).agg({'NBQUANTITE': 'count', 'PRESTATIONS_BRUTES': 'sum'}).reset_index())

### Total number of prescriptions, total amount by year and sous_categorie_dispensateur

In [ ]:
nb_and_amount_year_dispensateur = optimize_memory_df(df_drug_aos.groupby(['uuid','ANNEE_TRAITEMENT','SOUS_CATEGORIE_DISPENSATEUR'], observed = True).agg({'NBQUANTITE': 'count', 'PRESTATIONS_BRUTES': 'sum'}).reset_index())

In [ ]:
nb_year_dispensateur_onehot = nb_and_amount_year_dispensateur.pivot(columns = 'SOUS_CATEGORIE_DISPENSATEUR', index = ['uuid','ANNEE_TRAITEMENT'], values = 'NBQUANTITE').astype('category').reset_index()
amount_year_dispensateur_onehot = nb_and_amount_year_dispensateur.pivot(columns = 'SOUS_CATEGORIE_DISPENSATEUR', index = ['uuid','ANNEE_TRAITEMENT'], values = 'PRESTATIONS_BRUTES').reset_index()

In [ ]:
# Loop through all columns in the DataFrame
for column_name in nb_year_dispensateur_onehot.columns:
    # Check if the column is of Categorical data type
    if pd.api.types.is_categorical_dtype(nb_year_dispensateur_onehot[column_name]):
        # Get current categories
        current_categories = nb_year_dispensateur_onehot[column_name].cat.categories
        
        # Add the new category (0) to the list of categories, if not already present
        updated_categories = current_categories.tolist()
        if 0 not in updated_categories:
            updated_categories.append(0)

        # Update the categories of the Categorical column
        nb_year_dispensateur_onehot[column_name] = nb_year_dispensateur_onehot[column_name].cat.set_categories(updated_categories)

In [ ]:
nb_year_dispensateur_onehot = nb_year_dispensateur_onehot.fillna(0)
amount_year_dispensateur_onehot = amount_year_dispensateur_onehot.fillna(0)

### ATCs by year

In [ ]:
netamount_patient = df_drug_aos.groupby(['uuid','ANNEE_TRAITEMENT','ATC3'], observed = True)['PRESTATIONS_BRUTES'].sum()

In [ ]:
# df_amount_by_atc = pd.DataFrame(df_drug_aos.groupby(['uuid','ANNEE_TRAITEMENT','ATC3']).PRESTATIONS_BRUTES.sum()).reset_index()

In [ ]:
import dask.dataframe as dd

# Convert the pandas DataFrame to a Dask DataFrame
# Since we can't pivot a Dask dataframe using multiple columns...we use a little trick to get a single column combining year and uuid
if not os.path.isfile(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_atc.parquet.gzip'):

    df_drug_aos['year_uuid'] = df_drug_aos['ANNEE_TRAITEMENT'].astype(str)+'_'+ df_drug_aos['uuid'].astype(str)
    ddf_drug_aos = dd.from_pandas(df_drug_aos, npartitions=3)
    ddf_drug_aos = ddf_drug_aos.categorize(columns=['ATC3','year_uuid'])

    df_amount_by_atc = ddf_drug_aos.pivot_table(index='year_uuid',
                                                columns='ATC3',
                                                values='PRESTATIONS_BRUTES',
                                                aggfunc='sum').reset_index()

    df_amount_by_atc = df_amount_by_atc.compute()

    df_amount_by_atc[['ANNEE_TRAITEMENT','uuid']] = df_amount_by_atc.year_uuid.str.split("_", n=1, expand=True)
    df_amount_by_atc.to_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_atc.parquet.gzip', compression = 'gzip')

else:
    df_amount_by_atc = pd.read_parquet(data_folder/'processed'/'Intermediate datasets'/'df_amount_by_atc.parquet.gzip')

In [ ]:
netamount_patient = df_drug_aos.groupby(['uuid','ANNEE_TRAITEMENT'], observed = True)['PRESTATIONS_BRUTES'].sum()
patient_drugs = df_drug_aos[['uuid','ATC3','ANNEE_TRAITEMENT']].drop_duplicates()
X = pd.get_dummies(patient_drugs['ATC3'])
patient_drugs = pd.concat([patient_drugs.drop('ATC3', axis = 1), X], axis=1)

In [ ]:
del X

In [ ]:
patient_drugs = patient_drugs.groupby(['uuid','ANNEE_TRAITEMENT'], observed = True).sum()

In [ ]:
patient_drugs['PRESTATIONS_BRUTES'] = netamount_patient

In [ ]:
patient_drugs = patient_drugs.reset_index()
patient_drugs['n_atc'] = patient_drugs.drop(['uuid','PRESTATIONS_BRUTES','ANNEE_TRAITEMENT'],axis = 1).sum(axis = 1)

In [ ]:
patient_drugs = pd.merge(patient_drugs, amount_year_dispensateur_onehot, on = ['uuid','ANNEE_TRAITEMENT'], how = 'left')
patient_drugs = pd.merge(patient_drugs, nb_year_dispensateur_onehot, on = ['uuid','ANNEE_TRAITEMENT'], how = 'left', suffixes = ('','_amount') )

In [ ]:
for column_name in patient_drugs.columns:
    # Check if the column is of Categorical data type
    if pd.api.types.is_categorical_dtype(patient_drugs[column_name]):
        patient_drugs[column_name] = patient_drugs[column_name].astype(float)

In [ ]:
patient_drugs = patient_drugs.groupby(['uuid','ANNEE_TRAITEMENT']).sum().reset_index()

In [ ]:
patient_drugs = pd.merge(patient_drugs, df_amount_by_atc, on = ['uuid','ANNEE_TRAITEMENT'], how = 'left', suffixes=['_ATC_N','_ATC_AMOUNT']).fillna(0)

In [ ]:
del df_amount_by_atc

### Quality checks before final concatenation

In [ ]:
patient_drugs.shape[0]

In [ ]:
patient_drugs.uuid.nunique()

In [ ]:
df_outcomes_prestation_temp.shape[0]

In [ ]:
# df_all_uuid_years_combi.uuid.nunique()

In [ ]:
df_outcomes_prestation_temp.uuid.nunique()

In [ ]:
patient_drugs_light = patient_drugs[['uuid','ANNEE_TRAITEMENT','n_atc','PRESTATIONS_BRUTES']]

In [ ]:
patient_drugs_light.columns = ['uuid','ANNEE_TRAITEMENT','n_atc','PRESTATIONS_BRUTES_ATC']

In [ ]:
patient_drugs = patient_drugs.rename(columns={'PRESTATIONS_BRUTES':'PRESTATIONS_BRUTES_ATC'})

## Combine Prestations & Prescriptions data

The basis of the dataset should be fully exhaustive, i.e. containing all possible combinations of uuid-YEAR for the years 2017 to 2021. Some people have no prescription, others no prestation within a year, that's usual...but we also have some people that were not insured for both insurance for certain years. These have been flagged thanks to the `group definition` step. 

In [ ]:
import itertools
ids = pd.concat([df_couverture_aos.uuid, df_couverture_lca.uuid]).drop_duplicates()
years = [2017,2018,2019,2020,2021]
id_years = list(itertools.product(ids, years))
df_all_uuid_years_combi = pd.DataFrame(id_years, columns=['uuid', 'NOANNEE'])

In [ ]:
df_all_uuid_years_combi = df_all_uuid_years_combi.merge(df_gps_exploded_excl_gp, on=["NOANNEE", "uuid"], how="left")
df_all_uuid_years_combi['gp'] = df_all_uuid_years_combi['gp'].map(dict_labels_gps)

#### We see that we have ~104,000 uuid-NOANNEE combinations that don't have a group. We will have to solve this at some point. Probably something associated with some combinations not being present even in the df_couverture df. 

One example `74bc0983-8527-444a-8dbe-e267e5530772`:
- Used LCA in 2017
- 2018, 2019 : No use of his/her LCA, not insured AOS
- 2020, 2021 : No insurance AT ALL

By elimination, we can define that rows with gp = NaN are the ones with no insurance that year.

In [ ]:
df_all_uuid_years_combi[df_all_uuid_years_combi.gp == 'No insurance']

In [ ]:
df_all_uuid_years_combi.loc[df_all_uuid_years_combi.gp.isnull(), 'gp'] = 'No insurance'

The first building block is the insurance coverage data, which should include everyone

In [ ]:
# Create a dictionary of Swiss cantons and their abbreviations
swiss_cantons = {
    "Aargau": "AG",
    "Appenzell Ausserrhoden": "AR",
    "Appenzell Innerrhoden": "AI",
    "Basel-Landschaft": "BL",
    "Basel-Stadt": "BS",
    "Bern": "BE",
    "Fribourg": "FR",
    "Genève": "GE",
    "Glarus": "GL",
    "Graubünden": "GR",
    "Jura": "JU",
    "Luzern": "LU",
    "Neuchâtel": "NE",
    "Nidwalden": "NW",
    "Obwalden": "OW",
    "St. Gallen": "SG",
    "Schaffhausen": "SH",
    "Schwyz": "SZ",
    "Solothurn": "SO",
    "Thurgau": "TG",
    "Ticino": "TI",
    "Uri": "UR",
    "Vaud": "VD",
    "Valais":"VS",
    "Zug": "ZG",
    "Zürich": "ZH"
}

In [ ]:
df_coverage_aos = df_couverture_aos[['uuid','NOANNEE','NBAGE','CDPHYSSEXE','CDLANGUE','MODELE','MTFRANCHISECOUV','CANTON_NAME']].drop_duplicates()
df_coverage_lca = df_couverture_lca[['uuid','NOANNEE','NBAGE','CDPHYSSEXE','CDLANGUE','CANTON_NAME']].drop_duplicates()

In [ ]:
# concatenate df_lca and df_aos
df_coverage = pd.concat([df_coverage_aos, df_coverage_lca])
df_coverage = df_coverage.sort_values(['uuid','NOANNEE','MODELE']).drop_duplicates(subset = ['uuid','NOANNEE'])

In [ ]:
df_coverage['CANTON_ACRONYM'] = df_coverage['CANTON_NAME'].map(swiss_cantons)

In [ ]:
df_coverage = pd.concat([df_coverage,
                              pd.get_dummies(df_coverage.CDPHYSSEXE, prefix = 'SEX'),
                              pd.get_dummies(df_coverage.CDLANGUE, prefix = 'LANG'),
                              pd.get_dummies(df_coverage.MODELE, prefix = 'MODEL'),
                              pd.get_dummies(df_coverage.CANTON_ACRONYM, prefix = 'CANTON_ACRONYM'),
                              pd.get_dummies(df_coverage.MTFRANCHISECOUV, prefix = 'DEDUCTIBLE',dtype=int)],axis = 1).drop_duplicates().reset_index(drop = True)

In [ ]:
# df_coverage = pd.concat([df_coverage,
#                               pd.get_dummies(df_coverage.MTFRANCHISECOUV, prefix = 'DEDUCTIBLE_',dtype=int)], axis=1)

In [ ]:
df_all_uuid_years_combi_coverage = pd.merge(df_all_uuid_years_combi, df_coverage, on = ['uuid','NOANNEE'], how = 'left')

In [ ]:
df_all_uuid_years_combi_coverage_prestations = pd.merge(df_all_uuid_years_combi_coverage, df_outcomes_prestation_temp.drop(['NOANNEE','gp'], axis = 1), left_on = ['uuid','NOANNEE'], right_on = ['uuid',"ANNEE_TRAITEMENT"], how = 'left')

In [ ]:
df_all_uuid_years_combi_coverage_prestations['PRESTATIONS_BRUTES_AOS'].sum()

In [ ]:
df_all_uuid_years_combi_coverage_prestations['PRESTATIONS_BRUTES_LCA'].sum()

ALL GOOD !

In [ ]:
df_all_uuid_years_combi_coverage_prestations_drugs = pd.merge(df_all_uuid_years_combi_coverage_prestations, patient_drugs, left_on = ['uuid','NOANNEE'], right_on=['uuid','ANNEE_TRAITEMENT'], suffixes = ('_prestation','_drug'), how = 'left')
# df_all_uuid_years_combi_coverage_prestations_drugs = pd.merge(df_all_uuid_years_combi_coverage_prestations, patient_drugs_light, left_on = ['uuid','NOANNEE'], right_on=['uuid','ANNEE_TRAITEMENT'], how = 'left')

In [ ]:
df_all_uuid_years_combi_coverage_prestations_drugs.PRESTATIONS_BRUTES_AOS.sum()

In [ ]:
df_all_uuid_years_combi_coverage_prestations_drugs.PRESTATIONS_BRUTES_LCA.sum()

### Check on memory usage

In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:20]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
del df_prestation_lca_1spe, df_ostheo, df_prestation_lca_1spe_1session_notax, df_prestation_lca_1spe_1session, df_n_by_lca_therapy, df_amount_by_lca_therapy

In [ ]:
del merged_df, df_outcomes_prestation_temp, df_prestation_aos, df_couverture_aos, df_couverture_lca, df_prestation_lca, df_all_uuid_years_combi_coverage_prestations, df_amount_by_souscat_disp, df_all_uuid_years_combi_coverage, amount_year_dispensateur_onehot, df_grouped_month_outpatient, df_grouped_month_inpatient

In [ ]:
patient_drugs = optimize_memory_df(patient_drugs)
df_all_uuid_years_combi_coverage_prestations_drugs = optimize_memory_df(df_all_uuid_years_combi_coverage_prestations_drugs)

## Combine patient data with address data

- **Done** : Handling people who moved within a year -> creates duplicates

**Strategies**

*1. Averaging*: You could take an average of the environmental variables for individuals who have moved during the year. This assumes the time spent at each address is roughly equal and that the environmental variables have a linear relationship with health outcomes. However, this approach may not fully capture the effects if the environmental differences between the addresses are significant or if the time spent at each location was highly unequal.

*2. Weighted Averaging*: If you have information about the duration of stay at each address within the year, you could assign weights to the environmental variables based on the proportion of time spent at each location. This would provide a more accurate reflection of the individual's environmental exposure over the year.

*3. Use the Most Recent Address*: Another approach might be to use the environmental variables from the most recent address, assuming that this is the most relevant data. This might be appropriate if you believe recent environmental exposure is more relevant to the health outcomes you're studying.

*4. Separate Rows for Each Address*: If the relationship between the environment and health is complex, it might make sense to treat each change of address as a separate observation. This will complicate your analysis, as you'll need to account for repeated measures of the same individuals, but it will allow you to examine the effect of environment in more detail.

*5. Data Stratification*: Stratify the data into multiple categories such as "Never moved", "Moved once", "Moved more than once" and then perform the analysis. This approach could bring out interesting insights, however it could also limit the statistical power of your analysis due to the reduction in sample size within each stratum.


**06.07.2023 : Chose option 2**

In [ ]:
# data_with_env_variables = pd.merge(df_all_uuid_years_combi_coverage_prestations_drugs, df_full_address, on = ['uuid','NOANNEE'], how = 'outer')

In [ ]:
df_addresses_w_access_pollution_ndvi_lst_sep.loc[df_addresses_w_access_pollution_ndvi_lst_sep.NOANNEE.isnull(), 'NOANNEE'] = 2017

In [ ]:
# Create a dataframe of all unique ID and YEAR combinations
id_years = pd.DataFrame({'uuid': df_addresses_w_access_pollution_ndvi_lst_sep['uuid'].unique().tolist() * len([2017, 2018, 2019, 2020, 2021]),
                         'NOANNEE': [2017, 2018, 2019, 2020, 2021] * len(df_addresses_w_access_pollution_ndvi_lst_sep['uuid'].unique())})
# id_years['first_move_date'] = id_years.NOANNEE.apply(lambda x: pd.Timestamp(year=x, month=1, day=10))
# Cross join the ID/YEAR dataframe with the DF_ADDRESS dataframe
df_address_complete = pd.merge(id_years, df_addresses_w_access_pollution_ndvi_lst_sep[['uuid','NOANNEE','date','address_id']], on =['uuid','NOANNEE'], how='left')

# Forward fill the missing addresses
df_address_complete['address_id'] = df_address_complete.groupby('uuid')['address_id'].ffill()

# Backward fill any remaining missing addresses
df_address_complete['address_id'] = df_address_complete.groupby('uuid')['address_id'].bfill()

### Create weighted average of env variables for people that moved within a year

In [ ]:
# Identify rows with a move within a year
df_move = df_address_complete[df_address_complete['date'].notna()]

In [ ]:
# Create extra rows for each move within a year
df_extra = df_move.copy()
df_extra['address_id'] = np.nan
df_extra['date'] = pd.to_datetime(df_extra['NOANNEE'].astype(str) + '-01-01')

In [ ]:
# Append these extra rows to our original DataFrame and sort the rows
df_address_complete = pd.concat([df_address_complete, df_extra])
df_address_complete.sort_values(['uuid', 'NOANNEE', 'date'], inplace=True)
df_address_complete = df_address_complete.reset_index(drop = True)

In [ ]:
df_address_complete['address_id'] = df_address_complete.groupby(['uuid'])['address_id'].ffill()
df_address_complete['address_id'] = df_address_complete.groupby(['uuid'])['address_id'].bfill()

In [ ]:
# Extract the month of move in
df_address_complete['month_moved_in'] = df_address_complete['date'].dt.month -1

In [ ]:
# The month of move out would be the month in which the next move happened
df_address_complete['month_moved_out'] = df_address_complete.groupby(['uuid', 'NOANNEE'])['month_moved_in'].shift(-1)

In [ ]:
# If there is no next move in the same year, then month of move out would be December
df_address_complete.loc[df_address_complete['month_moved_in'].isna(), 'month_moved_in'] = 0
df_address_complete.loc[df_address_complete['month_moved_out'].isna(), 'month_moved_out'] = 12

In [ ]:
# Calculate the months at each address
df_address_complete['months_at_address'] = df_address_complete['month_moved_out'] - df_address_complete['month_moved_in']

# Calculate the weight
df_address_complete['weight'] = df_address_complete['months_at_address'] / 12.0

In [ ]:
# Group the dataframe by 'uuid' and 'NOANNEE', sum the weights, and reset the index
weight_check = df_address_complete.groupby(['uuid', 'NOANNEE'])['weight'].sum().reset_index()

# Check if any weight sum for each individual per year is not close to 1 (considering a small numerical tolerance)
incorrect_weights = weight_check[~np.isclose(weight_check['weight'], 1, atol=0.01)]

# Print the result
if incorrect_weights.empty:
    print("All individuals have weights summing to approximately 1 for each year.")
else:
    print(f"There are {len(incorrect_weights)} individuals-years with weights not summing to approximately 1.")
    print(incorrect_weights)

### Merge all data

In [ ]:
df_address_complete_w_env = pd.merge(df_address_complete, df_addresses_w_access_pollution_ndvi_lst_sep.drop_duplicates('address_id').drop(['uuid','ID_LAMAL','ID_LCA','NOANNEE','date','distance','doubl','MIN_of_Date_adress','distance_join_ndvi_lst'], axis = 1), on = 'address_id', how = 'left')

In [ ]:
env_columns = ['D_BAKERY', 'D_BOOK', 'D_GARAGE', 'D_GROCERY',
       'D_HMARKET', 'D_KIOSK', 'D_PHARMA', 'D_SMARKET', 'D_STORE', 'D_RESTO',
       'D_BANK', 'D_ADMIN', 'D_RETIRE', 'D_SECURITY', 'D_EDUC', 'D_SCHOOL_O',
       'D_SCHOOL_S', 'D_HAIR', 'D_SPORT', 'D_DENTIST', 'D_MEDIC', 'D_MEDIC_B',
       'D_MEDIC_S', 'D_STOP_0', 'D_STOP_1', 'D_STOP_2', 'D_STOP_3', 'D_STOP_4',
       'D_STOP_5', 'D_STOP_TOT', 'D_MUSEUM', 'D_FOREST', 'D_LAKE', 'D_SWIM',
       'D_ZOO', 'distance_join_access', 'mean_pm10', 'median_pm10',
       'mean_pm25', 'median_pm25', 'mean_no2', 'median_no2', 'mean_carday',
       'median_carday', 'mean_carnight', 'median_carnight',
       'distance_join_pollution', 'min_ndvi', 'max_ndvi', 'mean_ndvi',
       'median_ndvi', 'min_lst', 'max_lst', 'mean_lst', 'median_lst', 'ssep2','ssep3']
first_columns = ['address_id', 'lon_masked', 'lat_masked','geometry','ssep2_d', 'ssep2_t', 'ssep2_q', 'ssep3_d', 'ssep3_t', 'ssep3_q']
# Prepare dictionary for aggregation
aggregations = {col: 'sum' for col in env_columns}
aggregations.update({col: 'last' for col in first_columns})

for col in env_columns:
     df_address_complete_w_env[col] = df_address_complete_w_env[col] * df_address_complete_w_env['weight']

#### Weight env variables by addresses within a year

If a person lived at two addresses within the same year, we weight the env values associated with each address based on the number of months spent at each location.

In [ ]:
df_address_complete_w_env_weighted = df_address_complete_w_env.sort_values(['uuid','NOANNEE','weight']).groupby(['uuid','NOANNEE']).agg(aggregations).reset_index()

In [ ]:
del df_address_complete_w_env

#### Merge with addresses

In [ ]:
full_dataset = pd.merge(df_all_uuid_years_combi_coverage_prestations_drugs, df_address_complete_w_env_weighted, on = ['uuid','NOANNEE'], how = 'left')

In [ ]:
del df_all_uuid_years_combi_coverage_prestations_drugs, df_address_complete_w_env_weighted

In [ ]:
full_dataset.PRESTATIONS_BRUTES_AOS.sum()

In [ ]:
full_dataset.PRESTATIONS_BRUTES_LCA.sum()

In [ ]:
full_dataset[full_dataset.ssep3.isnull() == True].PRESTATIONS_BRUTES_AOS.sum()

#### Delete the few NA values for the SES index

In [ ]:
full_dataset_nonull = full_dataset[full_dataset.ssep3.isnull() == False]

In [ ]:
full_dataset_nonull.PRESTATIONS_BRUTES_LCA.sum()

In [ ]:
full_dataset_nonull.PRESTATIONS_BRUTES_AOS.sum()

In [ ]:
del full_dataset

In [ ]:
full_dataset_nonull.loc[:, 'A01A_ATC_N':'V10X_ATC_N'] = full_dataset_nonull.loc[:, 'A01A_ATC_N':'V10X_ATC_N'].fillna(0)
full_dataset_nonull.loc[:, 'A01A_ATC_AMOUNT':'V10X_ATC_AMOUNT'] = full_dataset_nonull.loc[:, 'A01A_ATC_AMOUNT':'V10X_ATC_AMOUNT'].fillna(0)
full_dataset_nonull.loc[:, 'Allergologie et immunologie clinique_prestation':'Urologie_prestation'] = full_dataset_nonull.loc[:, 'Allergologie et immunologie clinique_prestation':'Urologie_prestation'].fillna(0)

In [ ]:
full_dataset_nonull['n_atc'] = full_dataset_nonull['n_atc'].fillna(0)
full_dataset_nonull['PRESTATIONS_BRUTES_ATC'] = full_dataset_nonull['PRESTATIONS_BRUTES_ATC'].fillna(0)
full_dataset_nonull['n_month_lca_by_patient'] = full_dataset_nonull['n_month_lca_by_patient'].fillna(0)
full_dataset_nonull = full_dataset_nonull.drop('ANNEE_TRAITEMENT_prestation', axis = 1)

In [ ]:
full_dataset_nonull = gpd.GeoDataFrame(full_dataset_nonull, geometry = full_dataset_nonull['geometry'])

In [ ]:
# data_w_gis_nonull['lon'], data_w_gis_nonull['lat'] = data_w_gis_nonull['geometry'].x, data_w_gis_nonull['geometry'].y

In [ ]:
# data_w_gis_nonull.drop('geometry', axis = 1).to_parquet('../Data/processed/data_w_gis_nonull.parquet.gzip', compression='gzip')

In [ ]:
full_dataset_nonull.gp.value_counts()

In [ ]:
full_dataset_nonull.PRESTATIONS_BRUTES_AOS.sum()/1000000

## Create a Chronic disease score

In [ ]:
df_cds = pd.read_csv('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/ATC - Chronic disease score')

df_cds['Chronic diseases'] = df_cds['Chronic diseases'].ffill()

df_atc = pd.read_csv('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/atc_index_clean.csv')

dict_atc = dict(zip(df_atc.atc, df_atc.nameen))

dict_atc_inv = {v: k for k, v in dict_atc.items()}

df_cds['ATC'] = df_cds['Medication classes'].str.lower().map(dict_atc_inv)

#### Modification to the CDS ATC categories

- Some ATC listed in the categories of the CDS are on the ATC level 2 and some on level 4. Thus, for level 2, we replaced the level 2 code (ex. M10) to the level 3 categories existing for that ATC. 

- For level 4 ATC, it is less simple, we don't have the granularity to select the specific ATC of interest. We could either : be less specific and take all the ATC level 4 corresponding sharing the same ATC level 3. Or we could just remove the level 4 ATC altogether.

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
df_cds.loc[df_cds.ATC == 'M05','ATC'] = 'M05B'
df_cds = df_cds[df_cds.ATC != 'A02X']

In [ ]:
predictors = ['NBAGE','DEDUCTIBLE_0.0', 'DEDUCTIBLE_100.0',
       'DEDUCTIBLE_200.0', 'DEDUCTIBLE_300.0', 'DEDUCTIBLE_400.0',
       'DEDUCTIBLE_500.0', 'DEDUCTIBLE_600.0', 'DEDUCTIBLE_1000.0',
       'DEDUCTIBLE_1500.0', 'DEDUCTIBLE_2000.0', 'DEDUCTIBLE_2500.0','SEX_F',
              'CANTON_ACRONYM_AG',
 'CANTON_ACRONYM_AI',
 'CANTON_ACRONYM_AR',
 'CANTON_ACRONYM_BE',
 'CANTON_ACRONYM_BL',
 'CANTON_ACRONYM_BS',
 'CANTON_ACRONYM_FR',
 'CANTON_ACRONYM_GE',
 'CANTON_ACRONYM_GL',
 'CANTON_ACRONYM_GR',
 'CANTON_ACRONYM_JU',
 'CANTON_ACRONYM_LU',
 'CANTON_ACRONYM_NE',
 'CANTON_ACRONYM_NW',
 'CANTON_ACRONYM_OW',
 'CANTON_ACRONYM_SG',
 'CANTON_ACRONYM_SH',
 'CANTON_ACRONYM_SO',
 'CANTON_ACRONYM_SZ',
 'CANTON_ACRONYM_TG',
 'CANTON_ACRONYM_TI',
 'CANTON_ACRONYM_UR',
 'CANTON_ACRONYM_VD',
 'CANTON_ACRONYM_VS',
 'CANTON_ACRONYM_ZG',
 'CANTON_ACRONYM_ZH','ssep2']
chronic_conditions = [i+ "_ATC_N" for i in df_cds.ATC.values.tolist()]

In [ ]:
chronic_conditions = ['A02A_ATC_N',
 'A02B_ATC_N', 
 'M05B_ATC_N',
 'L01A_ATC_N',
 'L01B_ATC_N',
 'L01C_ATC_N',
 'L01D_ATC_N',
 'L01E_ATC_N',
 'L01F_ATC_N',
 'L01X_ATC_N',
 'C09A_ATC_N',
 'C09B_ATC_N',
 'C09C_ATC_N',
 'C09D_ATC_N',
 'C09X_ATC_N',
 'C02A_ATC_N',
# 'C02B_ATC_N',
 'C02C_ATC_N',
 'C02D_ATC_N',
 'C02K_ATC_N',
# 'C02L_ATC_N',
# 'C02N_ATC_N',
 'C04A_ATC_N',
 'C07A_ATC_N',
 'C08C_ATC_N',
 'C08D_ATC_N',
# 'C08E_ATC_N',
# 'C08G_ATC_N',
 'B01A_ATC_N',
 'N06D_ATC_N',
 'A10A_ATC_N',
 'A10B_ATC_N',
# 'A10X_ATC_N',
 'N03A_ATC_N',
 'S01E_ATC_N',
 'M04A_ATC_N',
 'J05A_ATC_N',
 'C10A_ATC_N',
 'C10B_ATC_N',
 'A07E_ATC_N',
 'B03A_ATC_N',
 'N02C_ATC_N',
 'N02A_ATC_N',
 'N02B_ATC_N',
 'N04A_ATC_N',
 'N04B_ATC_N',
# 'N04C_ATC_N',
 'N05B_ATC_N',
 'N05C_ATC_N',
 'N06A_ATC_N',
 'N05A_ATC_N',
 'R03A_ATC_N',
 'R03B_ATC_N',
 'R03C_ATC_N',
 'R03D_ATC_N',
 'M01A_ATC_N',
# 'M01B_ATC_N',
 'M01C_ATC_N',
 'M02A_ATC_N',
 'L04A_ATC_N',
 'H03A_ATC_N',
 'J04A_ATC_N']

In [ ]:
dict_chronic_diseases_atc = {
    "A02A_ATC_N":'Acid related disorders',
    "A02B_ATC_N":'Acid related disorders',
    'M05B_ATC_N':'Osteoporosis',
    'A10A_ATC_N':'Diabetes mellitus',
    'A10B_ATC_N':'Diabetes mellitus',
    'B03A_ATC_N':'Iron deficiency anemia',
    'N03A_ATC_N': 'Epilepsy',
    'C02A_ATC_N':'Cardiovascular diseases',
    'C02C_ATC_N':'Cardiovascular diseases',
    'C02D_ATC_N':'Cardiovascular diseases',
    'C02K_ATC_N':'Cardiovascular diseases',
    'C04A_ATC_N':'Cardiovascular diseases',
    'C07A_ATC_N':'Cardiovascular diseases',
    'C08C_ATC_N':'Cardiovascular diseases',
    'C08D_ATC_N':'Cardiovascular diseases',
    'C09A_ATC_N':'Cardiovascular diseases',
    'C09B_ATC_N':'Cardiovascular diseases',
    'C09C_ATC_N':'Cardiovascular diseases',
    'C09D_ATC_N':'Cardiovascular diseases',
    "C10A_ATC_N":"Hyperlipidemia",
    "C10B_ATC_N":"Hyperlipidemia",
    'L01A_ATC_N':'Cancer',
    'L01B_ATC_N':'Cancer',
    'L01C_ATC_N':'Cancer',
    'L01D_ATC_N':'Cancer',
    'L01E_ATC_N':'Cancer',
    'L01F_ATC_N':'Cancer',
    'L01X_ATC_N':'Cancer',
    "M01A_ATC_N":'Rheumatologic conditions',
    "M01C_ATC_N":'Rheumatologic conditions',
    "M02A_ATC_N":'Rheumatologic conditions',
    "M04A_ATC_N":'Gout and hyperuricemia',
    'N02A_ATC_N':'Pain',
    'N02B_ATC_N':"Pain",
    "N02C_ATC_N":'Migraines',
    "N04A_ATC_N":"Parkinson's disease",
    "N04B_ATC_N":"Parkinson's disease",
    'N05A_ATC_N':'Psychoses',

    'N05B_ATC_N':'Psychological disorders',
    'N05C_ATC_N':'Psychological disorders',
    'N06A_ATC_N':'Psychological disorders',

    "N06D_ATC_N":'Dementia',
    "L04A_ATC_N":'Rheumatologic conditions',
    "H03A_ATC_N":'Thyroid disorders',
    "J04A_ATC_N":'Tuberculosis',
    'J05A_ATC_N':'HIV',
    "R03A_ATC_N":'Respiratory illness',
    "S01E_ATC_N":'Glaucoma'
}

dict_chronic_diseases_atc_amount = {
    "A02A_ATC_AMOUNT":'Acid related disorders',
    "A02B_ATC_AMOUNT":'Acid related disorders',
    'M05B_ATC_AMOUNT':'Osteoporosis',
    'A10A_ATC_AMOUNT':'Diabetes mellitus',
    'A10B_ATC_AMOUNT':'Diabetes mellitus',
    'B03A_ATC_AMOUNT':'Iron deficiency anemia',
    'N03A_ATC_AMOUNT': 'Epilepsy',
    'C02A_ATC_AMOUNT':'Cardiovascular diseases',
    'C02C_ATC_AMOUNT':'Cardiovascular diseases',
    'C02D_ATC_AMOUNT':'Cardiovascular diseases',
    'C02K_ATC_AMOUNT':'Cardiovascular diseases',
    'C04A_ATC_AMOUNT':'Cardiovascular diseases',
    'C07A_ATC_AMOUNT':'Cardiovascular diseases',
    'C08C_ATC_AMOUNT':'Cardiovascular diseases',
    'C08D_ATC_AMOUNT':'Cardiovascular diseases',
    'C09A_ATC_AMOUNT':'Cardiovascular diseases',
    'C09B_ATC_AMOUNT':'Cardiovascular diseases',
    'C09C_ATC_AMOUNT':'Cardiovascular diseases',
    'C09D_ATC_AMOUNT':'Cardiovascular diseases',
    "C10A_ATC_AMOUNT":"Hyperlipidemia",
    "C10B_ATC_AMOUNT":"Hyperlipidemia",
    'L01A_ATC_AMOUNT':'Cancer',
    'L01B_ATC_AMOUNT':'Cancer',
    'L01C_ATC_AMOUNT':'Cancer',
    'L01D_ATC_AMOUNT':'Cancer',
    'L01E_ATC_AMOUNT':'Cancer',
    'L01F_ATC_AMOUNT':'Cancer',
    'L01X_ATC_AMOUNT':'Cancer',
    "M01A_ATC_AMOUNT":'Rheumatologic cond«itions',
    "M01C_ATC_AMOUNT":'Rheumatologic conditions',
    "M02A_ATC_AMOUNT":'Rheumatologic conditions',
    "M04A_ATC_AMOUNT":'Gout and hyperuricemia',
    'N02A_ATC_AMOUNT':'Pain',
    'N02B_ATC_AMOUNT':"Pain",
    "N02C_ATC_AMOUNT":'Migraines',
    "N04A_ATC_AMOUNT":"Parkinson's disease",
    "N04B_ATC_AMOUNT":"Parkinson's disease",
    'N05A_ATC_AMOUNT':'Psychoses',
    'N05B_ATC_AMOUNT':'Psychological disorders',
    'N05C_ATC_AMOUNT':'Psychological disorders',
    'N06A_ATC_AMOUNT':'Psychological disorders',
    "N06D_ATC_AMOUNT":'Dementia',
    "L04A_ATC_AMOUNT":'Rheumatologic conditions',
    "H03A_ATC_AMOUNT":'Thyroid disorders',
    "J04A_ATC_AMOUNT":'Tuberculosis',
    'J05A_ATC_AMOUNT':'HIV',
    "R03A_ATC_AMOUNT":'Respiratory illness',

    "S01E_ATC_AMOUNT":'Glaucoma',
}

In [ ]:
# Reverse the dictionary for grouping by disease
disease_to_atc = {}
for atc, disease in dict_chronic_diseases_atc_amount.items():
    if disease in disease_to_atc:
        disease_to_atc[disease].append(atc)
    else:
        disease_to_atc[disease] = [atc]

# For each disease, create a new column and check if any ATC columns are 1
for disease, atcs in disease_to_atc.items():
    full_dataset_nonull[disease] = full_dataset_nonull[atcs].sum(axis=1)

# If you want binary 0/1 values
# full_dataset_nonull = full_dataset_nonull.replace({disease: {2: 1 for disease in disease_to_atc.keys()}})

## Define treated and control groups

In [ ]:
full_dataset_nonull.loc[full_dataset_nonull.gp.isin(['LCA & AOS','LCA only']), 'treatment'] = 1
full_dataset_nonull.loc[full_dataset_nonull.gp.isin(['AOS only','No usage']), 'treatment'] = 0

In [ ]:
# full_dataset_nonull.loc[(full_dataset_nonull.PRESTATIONS_BRUTES_AOS > 0) &(full_dataset_nonull.PRESTATIONS_BRUTES_LCA > 0)&(full_dataset_nonull.PRESTATIONS_BRUTES_CAM > 0), 'treatment_lca_cam'] = 1
# full_dataset_nonull.loc[(full_dataset_nonull.PRESTATIONS_BRUTES_AOS > 0) &(full_dataset_nonull.PRESTATIONS_BRUTES_LCA == 0)&(full_dataset_nonull.PRESTATIONS_BRUTES_CAM == 0), 'treatment_lca_cam'] = 0


full_dataset_nonull['treatment_lca_cam'] = full_dataset_nonull['treatment'].copy()
full_dataset_nonull.loc[full_dataset_nonull.PRESTATIONS_BRUTES_CAM > 0, 'treatment_lca_cam'] = 1

In [ ]:
full_dataset_nonull.loc[(full_dataset_nonull.PRESTATIONS_BRUTES_AOS > 0) &(full_dataset_nonull.PRESTATIONS_BRUTES_LCA == 0)&(full_dataset_nonull.PRESTATIONS_BRUTES_CAM > 0), 'treatment_cam'] = 1
full_dataset_nonull.loc[(full_dataset_nonull.PRESTATIONS_BRUTES_AOS > 0) &(full_dataset_nonull.PRESTATIONS_BRUTES_LCA == 0)&(full_dataset_nonull.PRESTATIONS_BRUTES_CAM == 0), 'treatment_cam'] = 0

In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:25]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

## Calculate chronic disease score (CDS)

In [ ]:
full_dataset_nonull_with_cds = full_dataset_nonull[full_dataset_nonull.gp.isin(['AOS only','LCA & AOS','LCA only','No usage'])]

In [ ]:
chronic_diseases = list(set(dict_chronic_diseases_atc.values()))

In [ ]:
# Define a thresholding function
def threshold(value, threshold=0):
    return 1 if value > threshold else 0

# Apply thresholding to multiple columns
thresholded_df = full_dataset_nonull[chronic_diseases].applymap(lambda x: threshold(x))

In [ ]:
# Update the original DataFrame with thresholded columns
full_dataset_nonull_with_cds[chronic_diseases] = thresholded_df

In [ ]:
thresholded_df = thresholded_df.add_suffix('_binary')

In [ ]:
full_dataset_nonull[thresholded_df.columns] = thresholded_df

In [ ]:
del thresholded_df

In [ ]:
yearly_predicted_costs = {}
for year in full_dataset_nonull_with_cds.NOANNEE.unique():
    print(year)
    df_cds_year = full_dataset_nonull_with_cds[full_dataset_nonull_with_cds.NOANNEE == year][predictors + chronic_diseases + ['PRESTATIONS_TOTAL']]
    
    # Create two-part regression models WITHOUT TRAIN AND TEST SETS
    X = df_cds_year[predictors + chronic_diseases]
    y = df_cds_year['PRESTATIONS_TOTAL']
    # Part 1: Logistic regression to determine the probability of incurring health care costs per patient/year
    log_reg = LogisticRegression(max_iter=1000)
    # X_train, X_test, y_train, y_test = train_test_split(data_2017[predictors + chronic_conditions], data_2017['PRESTATIONS_TOTAL'], test_size=0.2, random_state=42)
    log_reg.fit(X, y > 0)
    print(log_reg.intercept_, log_reg.coef_, log_reg.score(X, y>0))

    prob_incurring_costs = log_reg.predict_proba(X)[:, 1]
#     df_cds_year['prob_cost'] = prob_incurring_costs
    # Part 2: Generalized linear regression with a gamma error distribution and linear link function
    # to estimate annual health care expenditures for patients incurring costs higher than 0
    df_cds_year['const'] = 1
    df_cds_year[predictors + chronic_diseases + ['const']]
    X_costs = X[y > 0]
    y_costs = y[y > 0]
    glm_gamma = sm.GLM(endog = y_costs, exog=X_costs, family=sm.families.Gamma(link=sm.families.links.identity()))
    glm_gamma_results = glm_gamma.fit()
    predicted_costs = glm_gamma_results.predict(X)
    expected_total_costs = prob_incurring_costs * predicted_costs
    glm_gamma_results.params[8:]

    chronic_disease_coef_dict = glm_gamma_results.params[40:].to_dict()

    weighted_sum = df_cds_year.apply(lambda x: x * chronic_disease_coef_dict[x.name] if x.name in chronic_disease_coef_dict.keys() else x * 0).sum(axis=1)

    yearly_predicted_costs[year] = weighted_sum

In [ ]:
glm_gamma_results.params[40:]

In [ ]:
print(glm_gamma_results.summary())

In [ ]:
cds = pd.DataFrame(pd.DataFrame.from_dict(yearly_predicted_costs).stack(), columns = ['CDS']).reset_index().drop('level_1', axis = 1).set_index('level_0')

In [ ]:
full_dataset_nonull['cds'] = cds

In [ ]:
full_dataset_nonull[['PRESTATIONS_BRUTES_LCA','PRESTATIONS_BRUTES_AOS','PRESTATIONS_NETTES_AOS','PRESTATIONS_BRUTES_CAM']].isna().sum().sort_values().tail(50)

In [ ]:
full_dataset_nonull[(full_dataset_nonull.PRESTATIONS_BRUTES_LCA.isnull())].groupby('gp').size()

In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
full_dataset_nonull.gp.value_counts()

In [ ]:
# Which can also be obtained through
# weights = data_2017[predictors + chronic_conditions + ['const']] * glm_gamma_results.params
# predicted_costs = weights.sum(axis = 1)
# expected_total_costs = prob_incurring_costs * predicted_costs

#### Export final dataset

- Features still missing:
    - Spike profile
    - Average monthly drug cost
    - Last three months cost (drug)
    - Last three months cost (prestation)
    - Number of months above mean (drug)
    - Number of months above mean (prestation)
    - Max monthly cost (drug)
    - Max monthly cost (prestation)

In [ ]:
full_dataset_nonull = full_dataset_nonull.rename(columns = {'PRESTATIONS_BRUTES':"DRUGAMOUNT_BRUT"})

#### Add age groups

In [ ]:
# Define bins for the age groups
bins = [0, 2, 6, 13, 19, 25, 35, 45, 65, 80, np.inf]
# Define labels for the age groups
labels = ['0-1', '2-5', '6-12', '13-18', '19-24', '25-34', '35-44', '45-64', '65-79', '80+']
# Create new column 'age_group' using pd.cut()
full_dataset_nonull['age_group'] = pd.cut(full_dataset_nonull['NBAGE'], bins=bins, labels=labels, right=False)

In [ ]:
full_dataset_nonull.to_parquet(data_folder/'processed'/'full_dataset_nonull.parquet.gzip', compression = 'gzip')

## Prevalence of state transitions

In [ ]:
test = df_outcomes_prestation[df_outcomes_prestation.ANNEE_TRAITEMENT == 2017]

In [ ]:
sankey_df = df_outcomes_prestation.pivot(columns = 'ANNEE_TRAITEMENT',index = 'uuid', values = 'gp')

In [ ]:
df_outcomes_prestation[df_outcomes_prestation.gp.isnull()].uuid.nunique()

In [ ]:
sankey_df['dummy'] = 1

In [ ]:
flows = (
    sankey_df.groupby([2017, 2018, 2019, 2020, 2021], observed = True)
    .agg({"dummy": "sum"})
    .dropna()
    .reset_index()
)

In [ ]:
t_intervals = [2017, 2018, 2019, 2020, 2021]
flows_full = pd.DataFrame()
for i in range(len(t_intervals) - 1):
    _ = (
        sankey_df.groupby([t_intervals[i],t_intervals[i+1]])
        .agg({"dummy": "sum"})
        .dropna()
        .reset_index()
    ).rename(
            columns={
                "dummy": "value",
                t_intervals[i]:'source',
                t_intervals[i+1]:'target'
            })
    _['year'] = t_intervals[i]

    flows_full = pd.concat([flows_full, _])

In [ ]:
flows_full[["source",'target']] = flows_full[["source",'target']].astype('category')

flows_full = flows_full.sort_values(['year','target','source'])

In [ ]:
eps = 0
for year in t_intervals:
    flows_full.loc[flows_full.year == year, 'source_code'] = flows_full[flows_full.year == year].source.cat.codes + eps
    flows_full.loc[flows_full.year == year, 'target_code'] = flows_full[flows_full.year == year].target.cat.codes + eps + 5
    eps+=5

In [ ]:
flows_fulls_labels = list(flows_full['source'])

In [ ]:
flows_full.loc[flows_full['target'].str.contains('LCA only'),'color'] = 'rgba(222,45,38, 0.4)'
flows_full.loc[flows_full['target'].str.contains('AOS only'),'color'] = 'rgba(49,130,189, 0.4)'
flows_full.loc[flows_full['target'].str.contains('LCA & AOS'),'color'] = 'rgba(173,221,142, 0.4)'
flows_full.loc[flows_full['target'].str.contains('No usage'),'color'] = 'rgba(189,189,189, 0.4)'
flows_full.loc[flows_full['target'].str.contains('Not insured LCA, AOS'),'color'] = 'rgba(189,189,189, 0.4)'
flows_full.loc[flows_full['target'].str.contains('Not insured AOS, LCA'),'color'] = 'rgba(189,189,189, 0.4)'

In [ ]:
colors = []
for i in flows_fulls_labels:
    if 'LCA only' in i:
        colors.append('rgba(222,45,38, 0.4)')
    elif 'AOS only' in i:
        colors.append('rgba(49,130,189, 0.4)')
    elif 'LCA & AOS' in i:
        colors.append('rgba(173,221,142, 0.4)')
    else:
        colors.append('rgba(189,189,189, 0.4)')

In [ ]:
import plotly.graph_objects as go
import urllib, json
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = " Individuals",
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  flows_fulls_labels,
      color = colors
    ),
    link = dict(
      source =  flows_full['source_code'],
      target =  flows_full['target_code'],
      value =  flows_full['value'],
      color = flows_full['color']))])

fig.update_layout(
    hovermode = 'x',
    title="Trajectoire des individus assurés entre 2017 et 2021",
    font=dict(size = 12, color = 'black'),
    plot_bgcolor='black',height = 500, width = 900,
    paper_bgcolor='white'
)
fig.write_html("../Results/fig1.html")


In [ ]:
fig.show()

## Seasonality and periodicity

<div class="alert alert-block alert-info">
<b>Problem Statement</b> 

1. Identify trends, periods and seasons in the monthly aggregated data
    
2. Given the monthly total LCA consumption from 2017 to 2020, forecast the monthly load in 2021. In general, the model shall be able to produce one-year-ahead forecasts.
</div>    

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

### 1. EDA

In [ ]:
sum_prestation_brutes_aos_by_patient_by_month = sum_prestation_brutes_aos_by_patient_by_month.groupby(['uuid','ANNEE_TRAITEMENT','MOIS_TRAITEMENT'])['PRESTATIONS_BRUTES'].sum()

In [ ]:
df_prestation_aos_outcomes_by_month = pd.concat([n_prestation_aos_by_patient_by_month,
                                                 sum_prestation_brutes_aos_by_patient_by_month,
                                                 sum_prestation_nettes_aos_by_patient_by_month,
                                                 n_bill_prestation_aos_by_patient_by_month], axis = 1).reset_index().rename(columns = {0:'NBROWS'})

In [ ]:
df_prestation_aos_outcomes_by_month['treatmentdate'] = pd.to_datetime(df_prestation_aos_outcomes_by_month['ANNEE_TRAITEMENT'].astype('string') + '-' + df_prestation_aos_outcomes_by_month['MOIS_TRAITEMENT'].astype('string'))
df_prestation_aos_outcomes_by_month['treatmentmonth'] = df_prestation_aos_outcomes_by_month['treatmentdate'].dt.strftime('%Y-%m')

In [ ]:
df_prestation_aos_outcomes_sums = df_prestation_aos_outcomes_by_month.groupby('treatmentdate', observed = True)[['NBRE_FACTURES','PRESTATIONS_BRUTES','PRESTATIONS_NETTES','NBROWS']].sum()

In [ ]:
df_prestation_aos_outcomes_sums = df_prestation_aos_outcomes_sums.asfreq('M', method = 'ffill')

In [ ]:
# Plot time-series of the PRESTATIONS_BRUTES variable
df_prestation_aos_outcomes_sums.PRESTATIONS_BRUTES.plot(ylim = (0, 100000000), figsize = (6,4))

Some features are visible:

- There was a big drop in the prestations during the 2020 lockdown
- The first months of each year seem to be the highest each time

We can further explore the trend with a moving average filter.

In [ ]:
# 3 months rolling window
df_prestation_aos_outcomes_sums.rolling(3).mean().PRESTATIONS_BRUTES.plot(ylim = (0, 100000000))

We can also have a look at the autocorrelation and partial autocorrelation function to check for the yearly seasonality.

In [ ]:
# plot the time series data
plt.plot(df_prestation_aos_outcomes_sums)
plt.xlabel('Date')
plt.ylabel('Value')
plt.show()

In [ ]:
# calculate the rolling mean and standard deviation
rolling_mean = df_prestation_aos_outcomes_sums.rolling(window=12).mean()
rolling_std = df_prestation_aos_outcomes_sums.rolling(window=12).std()

In [ ]:
# plot the rolling mean and standard deviation
plt.plot(rolling_mean, label=['Rolling Mean - Nbre factures','Rolling Mean - Montant Brut','Rolling Mean - Montant Net','Rolling Mean - Quantité'])
plt.legend()
plt.show()

In [ ]:
plt.plot(rolling_std, label=['Rolling Std - Nbre factures','Rolling Std - Montant Brut','Rolling Mean - Montant Net','Rolling Std - Quantité'])
plt.legend()


The `seasonal_decompose()` function from the `statsmodels.tsa.seasonal` library is used to decompose a time series into its trend, seasonal, and residual components. This function returns a `seasonal_decompose()` object, which includes the following attributes:

`trend`: the estimated trend component of the time series. <br/>
`seasonal`: the estimated seasonal component of the time series.<br/>
`resid`: the residuals of the time series after removing the trend and seasonal components.<br/>
`observed`: the original time series.<br/>

When interpreting the results from the `seasonal_decompose()` function, you should consider the following:

**Trend**: The trend component represents the long-term changes in the time series data. It can be used to identify whether the time series is increasing, decreasing, or staying relatively constant over time.

**Seasonality**: The seasonal component represents the repeating patterns in the time series data. It can be used to identify whether the time series has a regular seasonality, such as daily, weekly, or yearly patterns.

**Residuals**: The residuals represent the random or irregular variations in the time series data that are not explained by the trend and seasonal components.

**Plotting the decomposition**: The function also provides a plot of the decomposition, which allows you to visualize the trend, seasonal, and residual components of the time series. The trend component is represented by a solid line, the seasonal component is represented by a dotted line, and the residuals are represented by a dashed line. The original time series is also plotted for comparison.

**Model selection**: The function allows you to select a model between additive and multiplicative, the additive model is used when the seasonal component is constant over time and the multiplicative model is used when the seasonal component is proportional to the trend component.

It's important to note that the decomposition might not always work properly, in some cases, the decomposition may not be able to separate the trend and seasonal components cleanly, or the residuals may not be random. In these cases, you may need to try different methods or pre-process the data before decomposing it.

In [ ]:
# decompose the time series into its trend, seasonal, and residual components
result = seasonal_decompose(df_prestation_aos_outcomes_sums['PRESTATIONS_BRUTES'], model='multiplicative')
result.plot()
plt.show()

In [ ]:
# decompose the time series into its trend, seasonal, and residual components
result = seasonal_decompose(df_prestation_aos_outcomes_sums['NBRE_FACTURES'], model='multiplicative')
result.plot()
plt.show()

In [ ]:
result.trend

The plot_acf() function from the statsmodels.graphics.tsaplots library is used to plot the autocorrelation function (ACF) of a time series. The ACF is a measure of the similarity between a time series and a lagged version of itself.

When interpreting the results of the plot_acf() function, you should look for patterns in the plot that indicate how strongly the time series is correlated with lagged versions of itself. Here are some things to consider when interpreting the plot:

Autocorrelation at lag 0: The value at lag 0 represents the correlation of the time series with itself, which is always 1.

Autocorrelation at positive lags: Positive lags represent the correlation of the time series with lagged versions of itself. Positive autocorrelation values indicate that the time series is positively correlated with lagged versions of itself.

Autocorrelation at negative lags: Negative lags represent the correlation of the time series with lagged versions of itself, but with the time series reversed. Negative autocorrelation values indicate that the time series is negatively correlated with lagged versions of itself.

Confidence intervals: The plot also includes the confidence intervals represented by the blue shaded area. This area represents the range of values that the autocorrelation would be expected to fall in if the true autocorrelation was zero. If the autocorrelation values fall outside this area, it indicates that the autocorrelation is statistically significant.

Cutoff point: A cutoff point is represented by a horizontal line on the plot, it indicates the level of significance. If the autocorrelation values fall above this line, it means that they are statistically significant.

A decaying or an increasing trend: A decaying trend in the plot indicates that the correlation between the time series and its lagged versions decreases as the lag increases. An increasing trend would indicate that the correlation increases as the lag increases.

Partial Autocorrelation: The plot_acf() also plot the partial autocorrelation function (PACF) on the same plot, the PACF is a measure of the correlation between a time series and a lagged version of itself, with the effects of intermediate lags removed. The PACF can help you identify which lag values are most important in explaining the correlation.



In [ ]:
# calculate the autocorrelation and partial autocorrelation
plot_acf(df_prestation_aos_outcomes_sums['PRESTATIONS_BRUTES'])
plot_pacf(df_prestation_aos_outcomes_sums['PRESTATIONS_BRUTES'])
plt.show()

In [ ]:
from scipy import signal

In [ ]:
# load the time series data into a numpy array
time_series = [df_prestation_aos_outcomes_sums['PRESTATIONS_BRUTES']]

# calculate the periodogram
f, Pxx_den = signal.periodogram(time_series)

In [ ]:
# plot the periodogram
plt.semilogy(f, Pxx_den[0])
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power Spectral Density')
plt.show()

A periodogram is a graphical representation of the frequency components of a time series. It shows the power of the signal at different frequencies, which allows you to identify any patterns that repeat at regular intervals.

When interpreting a periodogram, you should look for peaks or troughs in the power spectral density (PSD) plot. These peaks or troughs indicate that certain frequencies are present in the time series. The height of the peak indicates the strength of the frequency component, and the width of the peak indicates how broad the frequency component is.

Here are some things to consider when interpreting a periodogram:

High frequency peaks: High frequency peaks indicate that there are rapid fluctuations in the time series data. These fluctuations are usually caused by noise or random variations in the data.

Low frequency peaks: Low frequency peaks indicate that there are slow trends or patterns in the time series data. These trends or patterns are usually caused by long-term changes or cyclical patterns in the data.

Multiple peaks: If there are multiple peaks in the periodogram, it indicates that there are multiple frequency components in the time series data. The presence of multiple peaks can also indicate that there are multiple underlying processes that are generating the data.

Sharp and narrow peaks: Sharp and narrow peaks indicate that the frequency component is well-defined and consistent.

Broad and low peaks: Broad and low peaks indicate that the frequency component is not well-defined and consistent.

It's important to note that the interpretation of the periodogram is highly dependent on the nature of the data, the sampling rate and the window applied. It's also important to consider if the data has been pre-processed in any way, as different pre-processing steps can affect the periodogram.